**Nota:** Este cuaderno acompaña a lo discutido en clases.

### RNN simple

In [ ]:
import torch
import torch.nn as nn

class SimpleRNNCell(nn.Module):
    """
    Una celda RNN simple con una capa totalmente conectada para la salida.
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNNCell, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_cell = nn.RNNCell(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de la celda RNN y la capa totalmente conectada.
        """
        nn.init.kaiming_uniform_(self.rnn_cell.weight_ih, nonlinearity='tanh')
        nn.init.kaiming_uniform_(self.rnn_cell.weight_hh, nonlinearity='tanh')
        nn.init.zeros_(self.rnn_cell.bias_ih)
        nn.init.zeros_(self.rnn_cell.bias_hh)
        
        nn.init.kaiming_uniform_(self.fc.weight, nonlinearity='linear')
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, hidden=None):
        """
        Paso hacia adelante de la celda RNN.
        
        Args:
            x (Tensor): Tensor de entrada en el paso de tiempo actual, forma (batch_size, input_size).
            hidden (Tensor, opcional): Estado oculto del paso de tiempo anterior, forma (batch_size, hidden_size).
                                       Si es None, se inicializa a ceros.
        
        Returns:
            output (Tensor): Tensor de salida en el paso de tiempo actual, forma (batch_size, output_size).
            hidden (Tensor): Estado oculto actualizado, forma (batch_size, hidden_size).
        """
        if hidden is None:
            hidden = torch.zeros(x.size(0), self.hidden_size, device=x.device, dtype=x.dtype)
        
        hidden = self.rnn_cell(x, hidden)
        output = self.fc(hidden)
        return output, hidden

if __name__ == "__main__":
    # Parámetros
    input_size = 10    # Dimensión de entrada de ejemplo
    hidden_size = 20   # Tamaño del estado oculto
    output_size = 5    # Dimensión de salida de ejemplo
    batch_size = 32    # Tamaño del lote de ejemplo
    
    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Inicializar la celda RNN
    rnn_cell = SimpleRNNCell(input_size, hidden_size, output_size).to(device)
    
    # Ejemplo de entrada y estado oculto
    x_t = torch.randn(batch_size, input_size, device=device)       # Entrada en el paso de tiempo t
    hidden_t = None  # Permitir que la celda RNN inicialice el estado oculto
    
    # Paso hacia adelante a través de la celda RNN
    y_t, hidden_t = rnn_cell(x_t, hidden_t)
    
    print("Salida en el paso de tiempo t:", y_t)
    print("Estado oculto en el paso de tiempo t:", hidden_t)


**Observaciones**

- Para entrenar este modelo, necesitarás definir una función de pérdida, un optimizador y un bucle de entrenamiento adecuado. Asegúrate de manejar correctamente la retropropagación a través del tiempo (BPTT) si trabajas con secuencias largas.

- Si tu objetivo es construir una red RNN completa para tareas como clasificación de secuencias o generación de texto, podrías considerar usar módulos más avanzados como nn.RNN, nn.LSTM o nn.GRU, que están optimizados y ofrecen funcionalidades adicionales.

- Para evitar el sobreajuste, podrías implementar técnicas de regularización como el dropout.

### RNN vista como una red neuronal feedforward

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class FeedforwardRNN(nn.Module):
    """
    Una Red Neuronal Recurrente (RNN) de tipo feedforward con una capa oculta.
    
    Args:
        input_size (int): Dimensionalidad de la entrada.
        hidden_size (int): Dimensionalidad del estado oculto.
        output_size (int): Dimensionalidad de la salida.
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedforwardRNN, self).__init__()
        # Definir matrices de pesos como capas lineales
        self.W = nn.Linear(input_size, hidden_size)    # Peso para x_t
        self.U = nn.Linear(hidden_size, hidden_size)   # Peso para h_{t-1}
        self.V = nn.Linear(hidden_size, output_size)   # Peso para h_t a y_t
        
        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de las capas lineales usando inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.W.weight)
        nn.init.zeros_(self.W.bias)
        
        nn.init.xavier_uniform_(self.U.weight)
        nn.init.zeros_(self.U.bias)
        
        nn.init.xavier_uniform_(self.V.weight)
        nn.init.zeros_(self.V.bias)

    def forward(self, x_t, h_t_minus_1=None):
        """
        Realiza un pase hacia adelante de la RNN.
        
        Args:
            x_t (Tensor): Entrada en el paso de tiempo actual, forma (batch_size, input_size).
            h_t_minus_1 (Tensor, opcional): Estado oculto del paso de tiempo anterior, forma (batch_size, hidden_size).
                                             Si es None, se inicializa a ceros.
        
        Returns:
            y_t (Tensor): Salida en el paso de tiempo actual, forma (batch_size, output_size).
            h_t (Tensor): Estado oculto actualizado, forma (batch_size, hidden_size).
        """
        if h_t_minus_1 is None:
            # Inicializar estado oculto a ceros si no se proporciona
            h_t_minus_1 = torch.zeros(x_t.size(0), self.U.out_features, device=x_t.device, dtype=x_t.dtype)
        
        # Calcular el nuevo estado oculto
        h_t = torch.tanh(self.W(x_t) + self.U(h_t_minus_1))
        
        # Calcular la salida
        y_t = self.V(h_t)
        
        return y_t, h_t

if __name__ == "__main__":
    # Parámetros
    input_size = 10    # Dimensión de entrada de ejemplo
    hidden_size = 20   # Tamaño del estado oculto
    output_size = 5    # Dimensión de salida de ejemplo
    batch_size = 32    # Tamaño del lote de ejemplo
    
    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")
    
    # Inicializar la RNN
    rnn = FeedforwardRNN(input_size, hidden_size, output_size).to(device)
    
    # Ejemplo de entrada y estado oculto
    x_t = torch.randn(batch_size, input_size, device=device)         # Entrada en el paso de tiempo t
    h_t_minus_1 = None  # Permitir que la RNN inicialice el estado oculto
    
    # Realizar un pase hacia adelante a través de la RNN
    y_t, h_t = rnn(x_t, h_t_minus_1)
    
    print("Salida y_t:", y_t)
    print("Estado oculto h_t:", h_t)


**Observaciones**

* Actualmente, la implementación maneja un solo paso de tiempo. Para trabajar con secuencias completas, necesitarás iterar sobre los pasos de tiempo y mantener el estado oculto a lo largo de la secuencia.
Considera encapsular el procesamiento de secuencias dentro de la clase o manejarlo externamente en el bucle de entrenamiento.
* Aunque estás implementando tu propia RNN, PyTorch ofrece módulos optimizados como nn.RNN, nn.LSTM y nn.GRU, que pueden ser más eficientes y ofrecer funcionalidades adicionales.
Estos módulos también manejan automáticamente el procesamiento de secuencias y la gestión de estados ocultos.
Regularización:
* Para evitar el sobreajuste, puedes implementar técnicas de regularización como dropout. PyTorch proporciona capas de dropout que pueden integrarse fácilmente en tu modelo.
Guardado y Carga del Modelo:
* Para guardar el estado del modelo y reanudar el entrenamiento posteriormente, puedes utilizar torch.save y torch.load.

### Desenrrollando una RNN en el tiempo

In [ ]:
import torch
import torch.nn as nn

class UnrolledRNN(nn.Module):
    """
    Una Red Neuronal Recurrente (RNN) desarrollada en el tiempo.
    
    Args:
        input_size (int): Dimensionalidad de la entrada.
        hidden_size (int): Dimensionalidad del estado oculto.
        output_size (int): Dimensionalidad de la salida.
        activation (callable, opcional): Función de activación para el estado oculto. Por defecto es tanh.
    """
    def __init__(self, input_size, hidden_size, output_size, activation=torch.tanh):
        super(UnrolledRNN, self).__init__()
        # Definir matrices de pesos como capas lineales
        self.W = nn.Linear(input_size, hidden_size)    # Peso para x_t
        self.U = nn.Linear(hidden_size, hidden_size)   # Peso para h_{t-1}
        self.V = nn.Linear(hidden_size, output_size)   # Peso para h_t a y_t
        self.activation = activation
        
        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de las capas lineales usando inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.W.weight)
        nn.init.zeros_(self.W.bias)
        
        nn.init.xavier_uniform_(self.U.weight)
        nn.init.zeros_(self.U.bias)
        
        nn.init.xavier_uniform_(self.V.weight)
        nn.init.zeros_(self.V.bias)

    def forward(self, inputs, h_0=None):
        """
        Realiza un pase hacia adelante de la RNN desarrollada en el tiempo.
        
        Args:
            inputs (Tensor): Secuencia de entradas, forma (seq_length, batch_size, input_size).
            h_0 (Tensor, opcional): Estado oculto inicial, forma (batch_size, hidden_size).
                                     Si es None, se inicializa a ceros.
        
        Returns:
            outputs (Tensor): Secuencia de salidas, forma (seq_length, batch_size, output_size).
            h_n (Tensor): Estado oculto final, forma (batch_size, hidden_size).
        """
        seq_length, batch_size, _ = inputs.size()
        
        if h_0 is None:
            # Inicializar estado oculto a ceros si no se proporciona
            h_t = torch.zeros(batch_size, self.U.out_features, device=inputs.device, dtype=inputs.dtype)
        else:
            h_t = h_0
        
        outputs = []
        
        # Desenrollar la RNN a través del tiempo
        for t in range(seq_length):
            x_t = inputs[t]
            # Calcular el nuevo estado oculto
            h_t = self.activation(self.W(x_t) + self.U(h_t))
            # Calcular la salida
            y_t = self.V(h_t)
            outputs.append(y_t.unsqueeze(0))  # Añadir una dimensión para el tiempo
        
        outputs = torch.cat(outputs, dim=0)  # Concatenar a lo largo del tiempo
        return outputs, h_t

if __name__ == "__main__":
    # Parámetros
    input_size = 10      # Dimensión de entrada de ejemplo
    hidden_size = 20     # Tamaño del estado oculto
    output_size = 5      # Dimensión de salida de ejemplo
    sequence_length = 3  # Longitud de la secuencia de entrada
    batch_size = 4       # Tamaño del lote de ejemplo
    
    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")
    
    # Inicializar la RNN
    rnn = UnrolledRNN(input_size, hidden_size, output_size).to(device)
    
    # Ejemplo de entrada y estado oculto inicial
    # Crear una secuencia de entradas: (seq_length, batch_size, input_size)
    inputs = torch.randn(sequence_length, batch_size, input_size, device=device)
    h_0 = torch.zeros(batch_size, hidden_size, device=device)  # Estado oculto inicial
    
    # Pase hacia adelante a través de la RNN
    outputs, h_n = rnn(inputs, h_0)
    
    # Imprimir salidas para cada paso de tiempo
    for t in range(sequence_length):
        print(f"Salida en el paso de tiempo {t+1}: {outputs[t]}")
    print("Estado oculto final:", h_n)


**Observaciones**

- Para entrenar este modelo, necesitarás definir una función de pérdida, un optimizador y un bucle de entrenamiento adecuado.  Si trabajas con secuencias más largas, considera optimizar el manejo de memoria y utilizar técnicas como truncated backpropagation through time (BPTT) para evitar problemas de memoria.
- Para evitar el sobreajuste, puedes implementar técnicas de regularización como dropout. PyTorch proporciona capas de dropout que pueden integrarse fácilmente en tu modelo.
- Para guardar el estado del modelo y reanudar el entrenamiento posteriormente, puedes utilizar torch.save y torch.load.
- Aunque estás implementando tu propia RNN, PyTorch ofrece módulos optimizados como nn.RNN, nn.LSTM y nn.GRU, que pueden ser más eficientes y ofrecer funcionalidades adicionales.
- Estos módulos también manejan automáticamente el procesamiento de secuencias y la gestión de estados ocultos.
- Para una comprensión más profunda, puedes visualizar cómo se actualizan los estados ocultos y las salidas a lo largo de los pasos de tiempo utilizando herramientas de visualización como TensorBoard.

### Modelo de lenguaje FFN y modelo de lenguaje RNN

In [ ]:
import torch
import torch.nn as nn

class FFNLanguageModel(nn.Module):
    """
    Modelo de Lenguaje basado en una Red Neuronal Feedforward (FFN).

    Este modelo toma tres entradas consecutivas (x_{t-2}, x_{t-1}, x_t}),
    las concatena y las procesa a través de una capa oculta para generar una
    salida y un estado oculto.

    Args:
        input_size (int): Dimensionalidad de cada entrada x_t.
        hidden_size (int): Dimensionalidad del estado oculto.
        output_size (int): Dimensionalidad de la salida y_t.
        activation (callable, opcional): Función de activación para la capa oculta. Por defecto es tanh.
    """
    def __init__(self, input_size, hidden_size, output_size, activation=torch.tanh):
        super(FFNLanguageModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.activation = activation

        # Definir capas lineales
        self.W = nn.Linear(input_size * 3, hidden_size)  # Para x_{t-2}, x_{t-1}, x_t
        self.U = nn.Linear(hidden_size, output_size)

        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de las capas lineales usando inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.W.weight)
        nn.init.zeros_(self.W.bias)
        
        nn.init.xavier_uniform_(self.U.weight)
        nn.init.zeros_(self.U.bias)

    def forward(self, x_t_minus_2, x_t_minus_1, x_t):
        """
        Realiza un pase hacia adelante del modelo FFN.

        Args:
            x_t_minus_2 (Tensor): Entrada en el tiempo t-2, forma (batch_size, input_size).
            x_t_minus_1 (Tensor): Entrada en el tiempo t-1, forma (batch_size, input_size).
            x_t (Tensor): Entrada en el tiempo t, forma (batch_size, input_size).

        Returns:
            y_t (Tensor): Salida en el tiempo t, forma (batch_size, output_size).
            h_t (Tensor): Estado oculto en el tiempo t, forma (batch_size, hidden_size).
        """
        # Concatenar las entradas
        x_concat = torch.cat((x_t_minus_2, x_t_minus_1, x_t), dim=1)
        # Calcular el estado oculto
        h_t = self.activation(self.W(x_concat))
        # Calcular la salida
        y_t = self.U(h_t)
        return y_t, h_t


class RNNLanguageModel(nn.Module):
    """
    Modelo de Lenguaje basado en una Red Neuronal Recurrente (RNN).

    Este modelo toma una entrada actual x_t y el estado oculto anterior h_{t-1},
    y genera una salida y_t y un nuevo estado oculto h_t.

    Args:
        input_size (int): Dimensionalidad de cada entrada x_t.
        hidden_size (int): Dimensionalidad del estado oculto.
        output_size (int): Dimensionalidad de la salida y_t.
        activation (callable, opcional): Función de activación para el estado oculto. Por defecto es tanh.
    """
    def __init__(self, input_size, hidden_size, output_size, activation=torch.tanh):
        super(RNNLanguageModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.activation = activation

        # Definir capas lineales
        self.W = nn.Linear(input_size, hidden_size)   # Para x_t
        self.U = nn.Linear(hidden_size, hidden_size)  # Para h_{t-1}
        self.V = nn.Linear(hidden_size, output_size)

        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de las capas lineales usando inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.W.weight)
        nn.init.zeros_(self.W.bias)
        
        nn.init.xavier_uniform_(self.U.weight)
        nn.init.zeros_(self.U.bias)
        
        nn.init.xavier_uniform_(self.V.weight)
        nn.init.zeros_(self.V.bias)

    def forward(self, x_t, h_t_minus_1=None):
        """
        Realiza un pase hacia adelante del modelo RNN.

        Args:
            x_t (Tensor): Entrada en el tiempo t, forma (batch_size, input_size).
            h_t_minus_1 (Tensor, opcional): Estado oculto anterior h_{t-1}, forma (batch_size, hidden_size).
                                           Si es None, se inicializa a ceros.

        Returns:
            y_t (Tensor): Salida en el tiempo t, forma (batch_size, output_size).
            h_t (Tensor): Nuevo estado oculto h_t, forma (batch_size, hidden_size).
        """
        if h_t_minus_1 is None:
            # Inicializar el estado oculto a ceros si no se proporciona
            h_t_minus_1 = torch.zeros(x_t.size(0), self.hidden_size, device=x_t.device, dtype=x_t.dtype)
        
        # Calcular el nuevo estado oculto
        h_t = self.activation(self.W(x_t) + self.U(h_t_minus_1))
        # Calcular la salida
        y_t = self.V(h_t)
        return y_t, h_t


def initialize_models(input_size, hidden_size, output_size, device):
    """
    Inicializa los modelos FFN y RNN y los mueve al dispositivo especificado.

    Args:
        input_size (int): Dimensionalidad de cada entrada x_t.
        hidden_size (int): Dimensionalidad del estado oculto.
        output_size (int): Dimensionalidad de la salida y_t.
        device (torch.device): Dispositivo (CPU o GPU) donde se alojarán los modelos.

    Returns:
        ffn_model (FFNLanguageModel): Modelo de lenguaje FFN inicializado.
        rnn_model (RNNLanguageModel): Modelo de lenguaje RNN inicializado.
    """
    ffn_model = FFNLanguageModel(input_size, hidden_size, output_size).to(device)
    rnn_model = RNNLanguageModel(input_size, hidden_size, output_size).to(device)
    return ffn_model, rnn_model


def main():
    # Parámetros
    input_size = 5    # Dimensión de cada x_t
    hidden_size = 10  # Dimensión del estado oculto h_t
    output_size = 3   # Dimensión de la salida y_t
    batch_size = 4    # Tamaño del lote

    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    # Inicializar los modelos
    ffn_model, rnn_model = initialize_models(input_size, hidden_size, output_size, device)

    # Ejemplo de entradas para el modelo FFN
    x_t_minus_2 = torch.randn(batch_size, input_size, device=device)
    x_t_minus_1 = torch.randn(batch_size, input_size, device=device)
    x_t = torch.randn(batch_size, input_size, device=device)

    # Estado oculto inicial para el modelo RNN
    h_t_minus_1 = torch.zeros(batch_size, hidden_size, device=device)

    # Pase hacia adelante a través del modelo FFN
    y_t_ffn, h_t_ffn = ffn_model(x_t_minus_2, x_t_minus_1, x_t)
    print("FFN salida y_t:\n", y_t_ffn)
    print("FFN estado oculto h_t:\n", h_t_ffn)

    # Pase hacia adelante a través del modelo RNN
    y_t_rnn, h_t_rnn = rnn_model(x_t, h_t_minus_1)
    print("RNN salida y_t:\n", y_t_rnn)
    print("RNN estado oculta h_t:\n", h_t_rnn)


if __name__ == "__main__":
    main()


La salida que estás viendo es el resultado de pasar datos a través de los modelos de lenguaje FFN y RNN que definimos anteriormente en PyTorch. 

1. **Usando dispositivo: cpu**: Este mensaje indica que el modelo está utilizando la CPU para la computación. Si tuvieras acceso a una GPU y configuraras el modelo para usarla, aquí verías "cuda" en lugar de "cpu".

2. **FFN salida $ y_t $**: Este tensor representa la salida $ y_t $ generada por el modelo de red neuronal feedforward (FFN) en cada uno de los pasos de tiempo. En este caso, tienes una salida para cada ejemplo en el lote (4 en total), con una dimensión de salida de 3 (especificada por el parámetro `output_size`).

   Ejemplo de salida:
   ```plaintext
   tensor([[ 0.6602,  0.4772, -0.5175],
           [-0.8437,  1.0741,  0.7102],
           [-0.4538,  0.2009,  0.1363],
           [-0.1967,  0.1131, -0.1583]], grad_fn=<AddmmBackward0>)
   ```
   Cada fila representa la salida $ y_t $ para una entrada diferente en el lote.

3. **FFN estado oculto $ h_t $**: Este tensor es el estado oculto $ h_t $ generado por el modelo FFN después de procesar las entradas concatenadas $ x_{t-2}, x_{t-1}, x_t $. Cada fila representa el estado oculto correspondiente a una entrada en el lote, y tiene una dimensión de 10 (especificada por `hidden_size`).

   Ejemplo de estado oculto:
   ```plaintext
   tensor([[ 0.5509,  0.8237,  0.3393,  0.3819, -0.9719,  0.9283, -0.0932, -0.4907,
             0.9283, -0.9370],
            ...], grad_fn=<TanhBackward0>)
   ```

4. **RNN salida $ y_t $**: Este tensor representa la salida $ y_t $ generada por el modelo RNN en cada uno de los pasos de tiempo. Similar a la salida del FFN, contiene una fila para cada entrada en el lote, con 3 valores de salida (especificado por `output_size`).

   Ejemplo de salida:
   ```plaintext
   tensor([[-0.2385,  0.0706,  0.4726],
           [ 0.3947, -0.2080,  1.6990],
           [-0.2079, -0.6942,  1.2398],
           [ 0.7105,  0.2177,  0.9162]], grad_fn=<AddmmBackward0>)
   ```

5. **RNN estado oculto $ h_t $**: Este tensor representa el estado oculto $ h_t $ actualizado en cada paso de tiempo del modelo RNN. Este estado se calcula usando tanto el estado oculto anterior $ h_{t-1} $ como la entrada actual $ x_t $, lo cual es característico de las redes RNN.

   Ejemplo de estado oculto:
   ```plaintext
   tensor([[ 0.5921,  0.8117,  0.4427, -0.1933,  0.9189,  0.0968,  0.1448,  0.2683,
             0.6134, -0.2545],
            ...], grad_fn=<TanhBackward0>)
   ```


- Cada salida $ y_t $ representa el valor de predicción del modelo para una determinada entrada. En el contexto de un modelo de lenguaje, estos podrían representar las probabilidades de las siguientes palabras en una secuencia.
- El estado oculto $ h_t $ es el "estado de memoria" de la red que ayuda a capturar información de las entradas anteriores (en el caso de la RNN) o de la concatenación de las entradas (en el caso del FFN).
- `grad_fn=<AddmmBackward0>` y `grad_fn=<TanhBackward0>` son funciones de gradiente generadas por PyTorch para facilitar la retropropagación durante el entrenamiento del modelo. Estas indican que los tensores son parte de un gráfico computacional que PyTorch utilizará para calcular gradientes y actualizar los parámetros del modelo.

### Entrenando RNN como modelos de lenguaje

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os

class RNNLanguageModel(nn.Module):
    """
    Modelo de Lenguaje basado en una Red Neuronal Recurrente (RNN).

    Este modelo utiliza una capa de embeddings para convertir tokens de entrada en vectores densos,
    una capa RNN para procesar secuencias de embeddings y una capa completamente conectada para
    predecir la siguiente palabra en la secuencia.

    Args:
        vocab_size (int): Tamaño del vocabulario (número de tokens únicos).
        embed_size (int): Dimensionalidad de los embeddings.
        hidden_size (int): Dimensionalidad del estado oculto de la RNN.
        num_layers (int): Número de capas de la RNN.
        dropout (float, opcional): Tasa de dropout entre capas de la RNN. Por defecto es 0.0.
    """
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.0):
        super(RNNLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)  # Capa de embeddings
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers=num_layers,
                          batch_first=True, dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Linear(hidden_size, vocab_size)  # Capa completamente conectada

        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de las capas lineales usando la inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        for name, param in self.rnn.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, hidden=None):
        """
        Realiza un pase hacia adelante del modelo RNN.

        Args:
            x (Tensor): Secuencia de entrada, forma (batch_size, seq_length).
            hidden (Tensor, opcional): Estado oculto inicial, forma (num_layers, batch_size, hidden_size).
                                       Si es None, se inicializa a ceros.

        Returns:
            logits (Tensor): Logits para cada posición de la secuencia, forma (batch_size, seq_length, vocab_size).
            hidden (Tensor): Estado oculto final, forma (num_layers, batch_size, hidden_size).
        """
        embeddings = self.embedding(x)  # (batch_size, seq_length, embed_size)
        output, hidden = self.rnn(embeddings, hidden)  # output: (batch_size, seq_length, hidden_size)
        logits = self.fc(output)  # (batch_size, seq_length, vocab_size)
        return logits, hidden

class DummyDataset(Dataset):
    """
    Dataset de ejemplo que genera secuencias aleatorias de tokens.

    Este dataset genera pares de secuencias de entrada y objetivo de manera aleatoria para fines de demostración.
    """
    def __init__(self, num_samples, seq_length, vocab_size):
        super(DummyDataset, self).__init__()
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.vocab_size = vocab_size

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Secuencia de entrada
        x = torch.randint(0, self.vocab_size, (self.seq_length,))
        # Secuencia objetivo (desplazada una posición hacia la derecha)
        y = torch.randint(0, self.vocab_size, (self.seq_length,))
        return x, y

def train(model, dataloader, criterion, optimizer, device, epoch, total_epochs, clip=5.0):
    """
    Función de entrenamiento para una época.

    Args:
        model (nn.Module): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los datos de entrenamiento.
        criterion (nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        device (torch.device): Dispositivo (CPU o GPU).
        epoch (int): Número de la época actual.
        total_epochs (int): Número total de épocas.
        clip (float, opcional): Valor máximo para el recorte de gradientes. Por defecto es 5.0.
    """
    model.train()
    epoch_loss = 0.0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{total_epochs}")

    for batch_idx, (inputs, targets) in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Mover datos al dispositivo

        optimizer.zero_grad()
        logits, _ = model(inputs)  # Forward pass

        # Reorganizar los logits y targets para calcular la pérdida
        logits = logits.view(-1, logits.size(-1))  # (batch_size * seq_length, vocab_size)
        targets = targets.view(-1)  # (batch_size * seq_length)

        loss = criterion(logits, targets)
        loss.backward()

        # Recorte de gradientes para evitar explosiones
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    average_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{total_epochs} - Loss: {average_loss:.4f}")

def evaluate(model, dataloader, criterion, device):
    """
    Función de evaluación para calcular la pérdida en el conjunto de validación.

    Args:
        model (nn.Module): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los datos de validación.
        criterion (nn.Module): Función de pérdida.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        float: Pérdida promedio en el conjunto de validación.
    """
    model.eval()
    epoch_loss = 0.0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            logits, _ = model(inputs)
            logits = logits.view(-1, logits.size(-1))
            targets = targets.view(-1)

            loss = criterion(logits, targets)
            epoch_loss += loss.item()

    average_loss = epoch_loss / len(dataloader)
    return average_loss

def save_model(model, optimizer, epoch, loss, path):
    """
    Guarda el estado del modelo y del optimizador.

    Args:
        model (nn.Module): Modelo a guardar.
        optimizer (torch.optim.Optimizer): Optimizador a guardar.
        epoch (int): Número de la época actual.
        loss (float): Pérdida actual.
        path (str): Ruta donde se guardará el modelo.
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path}")

def load_model(model, optimizer, path, device):
    """
    Carga el estado del modelo y del optimizador desde un archivo.

    Args:
        model (nn.Module): Modelo a cargar.
        optimizer (torch.optim.Optimizer): Optimizador a cargar.
        path (str): Ruta desde donde se cargará el modelo.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        int: Época desde la cual se reanudó el entrenamiento.
        float: Pérdida en la última época registrada.
    """
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

def main():
    # Parámetros
    vocab_size = 10000    # Tamaño del vocabulario
    embed_size = 128      # Dimensionalidad de los embeddings
    hidden_size = 256     # Dimensionalidad del estado oculto
    num_layers = 2        # Número de capas de la RNN
    dropout = 0.5         # Tasa de dropout
    sequence_length = 5   # Longitud de la secuencia
    batch_size = 32       # Tamaño del lote
    num_epochs = 10       # Número de épocas
    learning_rate = 0.001 # Tasa de aprendizaje
    save_path = 'rnn_language_model.pth'  # Ruta para guardar el modelo

    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    # Inicializar el modelo
    model = RNNLanguageModel(vocab_size, embed_size, hidden_size, num_layers, dropout).to(device)

    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Crear datasets y dataloaders
    train_dataset = DummyDataset(num_samples=10000, seq_length=sequence_length, vocab_size=vocab_size)
    val_dataset = DummyDataset(num_samples=2000, seq_length=sequence_length, vocab_size=vocab_size)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(save_path):
        start_epoch, _ = load_model(model, optimizer, save_path, device)

    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        train(model, train_loader, criterion, optimizer, device, epoch, num_epochs)
        val_loss = evaluate(model, val_loader, criterion, device)
        print(f"Epoca {epoch+1}/{num_epochs} - Validacion cruzada: {val_loss:.4f}")

        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch + 1, val_loss, save_path)

if __name__ == "__main__":
    main()


**Observaciones:**

- La clase DummyDataset genera datos aleatorios para propósitos de demostración. En un escenario real, deberías reemplazarla con un dataset que contenga secuencias de texto tokenizadas.
Puedes utilizar datasets como Penn Treebank o cualquier otro corpus de texto.
- Para trabajar con secuencias más largas, considera ajustar el sequence_length y manejar eficientemente la memoria.
Técnicas como truncated backpropagation through time (BPTT) pueden ser útiles para evitar problemas de memoria y mejorar el entrenamiento en secuencias largas.
- Se ha añadido una capa de dropout en la RNN para prevenir el sobreajuste. Puedes ajustar la tasa de dropout según sea necesario.
- Además de la pérdida, puedes implementar métricas como accuracy para monitorear el rendimiento del modelo. Esto proporciona una visión más completa del comportamiento del modelo durante el entrenamiento y la evaluación.
- Experimenta con diferentes tasas de aprendizaje, tamaños de lote, dimensiones de embeddings y tamaños ocultos para encontrar la mejor configuración para tu tarea específica.
- Considera utilizar variantes más avanzadas de RNN, como LSTM o GRU, que manejan mejor el problema del desvanecimiento de gradientes y capturan dependencias a largo plazo de manera más efectiva.
- PyTorch facilita la implementación de estos modelos utilizando nn.LSTM o nn.GRU.

Este ejemplo utiliza el dataset Penn Treebank, pero puedes adaptarlo!.

In [ ]:
from torchtext.datasets import PennTreebank
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

class TextDataset(Dataset):
    """
    Dataset para datos de texto que maneja la tokenización y la creación del vocabulario.
    """
    def __init__(self, split, tokenizer, vocab, seq_length):
        self.data = []
        for item in split:
            tokens = tokenizer(item)
            token_ids = vocab(tokens)
            self.data.extend(token_ids)
        self.seq_length = seq_length

    def __len__(self):
        return (len(self.data) - 1) // self.seq_length

    def __getitem__(self, idx):
        start = idx * self.seq_length
        end = start + self.seq_length
        x = torch.tensor(self.data[start:end], dtype=torch.long)
        y = torch.tensor(self.data[start+1:end+1], dtype=torch.long)
        return x, y

def build_vocab():
    """
    Construye el vocabulario a partir del dataset de entrenamiento.
    """
    tokenizer = get_tokenizer('basic_english')
    train_iter = PennTreebank(split='train')
    vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=["<unk>"])
    vocab.set_default_index(vocab["<unk>"])
    return vocab

def main_real_data():
    # Parámetros
    vocab_size = 10000    # Tamaño del vocabulario
    embed_size = 128      # Dimensionalidad de los embeddings
    hidden_size = 256     # Dimensionalidad del estado oculto
    num_layers = 2        # Número de capas de la RNN
    dropout = 0.5         # Tasa de dropout
    sequence_length = 30  # Longitud de la secuencia
    batch_size = 64       # Tamaño del lote
    num_epochs = 20       # Número de épocas
    learning_rate = 0.001 # Tasa de aprendizaje
    save_path = 'rnn_language_model_real_data.pth'  # Ruta para guardar el modelo

    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    # Construir el vocabulario
    vocab = build_vocab()
    actual_vocab_size = len(vocab)
    print(f"Tamaño del vocabulario: {actual_vocab_size}")

    # Cargar los datos de texto
    tokenizer = get_tokenizer('basic_english')
    train_iter = PennTreebank(split='train')
    val_iter = PennTreebank(split='valid')

    train_dataset = TextDataset(train_iter, tokenizer, vocab, sequence_length)
    val_dataset = TextDataset(val_iter, tokenizer, vocab, sequence_length)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Inicializar el modelo
    model = RNNLanguageModel(actual_vocab_size, embed_size, hidden_size, num_layers, dropout).to(device)

    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(save_path):
        start_epoch, _ = load_model(model, optimizer, save_path, device)

    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        train(model, train_loader, criterion, optimizer, device, epoch, num_epochs)
        val_loss = evaluate(model, val_loader, criterion, device)
        print(f"Epoch {epoch+1}/{num_epochs} - Validación Loss: {val_loss:.4f}")

        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch + 1, val_loss, save_path)

if __name__ == "__main__":
    main()
    # Para usar datos reales, comenta la línea anterior y descomenta la siguiente:
    # main_real_data()


### POS como etiquetado de secuencia

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os

class POSTaggerRNN(nn.Module):
    """
    Modelo de Etiquetado POS basado en una Red Neuronal Recurrente (RNN).

    Este modelo utiliza una capa de embeddings para convertir tokens de entrada en vectores densos,
    una capa LSTM para procesar secuencias de embeddings y una capa completamente conectada para
    predecir la etiqueta POS de cada palabra en la secuencia.

    Args:
        vocab_size (int): Tamaño del vocabulario (número de tokens únicos).
        embed_size (int): Dimensionalidad de los embeddings.
        hidden_size (int): Dimensionalidad del estado oculto de la LSTM.
        num_tags (int): Número de etiquetas POS.
        num_layers (int, opcional): Número de capas de la LSTM. Por defecto es 1.
        dropout (float, opcional): Tasa de dropout entre capas de la LSTM. Por defecto es 0.0.
    """
    def __init__(self, vocab_size, embed_size, hidden_size, num_tags, num_layers=1, dropout=0.0):
        super(POSTaggerRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)  # Capa de embeddings
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0.0)  # Capa LSTM
        self.fc = nn.Linear(hidden_size, num_tags)  # Capa completamente conectada

        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de las capas lineales y de embeddings usando la inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x):
        """
        Realiza un pase hacia adelante del modelo RNN para etiquetado POS.

        Args:
            x (Tensor): Secuencia de entrada, forma (batch_size, seq_length).

        Returns:
            tag_scores (Tensor): Puntuaciones para cada etiqueta POS en cada posición, forma (batch_size, seq_length, num_tags).
        """
        embeddings = self.embedding(x)  # (batch_size, seq_length, embed_size)
        lstm_out, _ = self.lstm(embeddings)  # (batch_size, seq_length, hidden_size)
        tag_scores = self.fc(lstm_out)  # (batch_size, seq_length, num_tags)
        return tag_scores

class DummyPOSTaggingDataset(Dataset):
    """
    Dataset de ejemplo para Etiquetado POS que genera secuencias aleatorias de tokens y etiquetas.

    Este dataset genera pares de secuencias de entrada y etiquetas de manera aleatoria para fines de demostración.
    """
    def __init__(self, num_samples, seq_length, vocab_size, num_tags):
        """
        Inicializa el dataset.

        Args:
            num_samples (int): Número de muestras en el dataset.
            seq_length (int): Longitud de cada secuencia.
            vocab_size (int): Tamaño del vocabulario.
            num_tags (int): Número de etiquetas POS.
        """
        super(DummyPOSTaggingDataset, self).__init__()
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.vocab_size = vocab_size
        self.num_tags = num_tags

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Secuencia de entrada (tokens)
        x = torch.randint(0, self.vocab_size, (self.seq_length,))
        # Secuencia de etiquetas POS (etiquetas)
        y = torch.randint(0, self.num_tags, (self.seq_length,))
        return x, y

def train(model, dataloader, criterion, optimizer, device, epoch, total_epochs, clip=5.0):
    """
    Función de entrenamiento para una época.

    Args:
        model (nn.Module): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los datos de entrenamiento.
        criterion (nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        device (torch.device): Dispositivo (CPU o GPU).
        epoch (int): Número de la época actual.
        total_epochs (int): Número total de épocas.
        clip (float, opcional): Valor máximo para el recorte de gradientes. Por defecto es 5.0.
    """
    model.train()
    epoch_loss = 0.0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{total_epochs}")

    for batch_idx, (inputs, targets) in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Mover datos al dispositivo

        optimizer.zero_grad()
        tag_scores = model(inputs)  # Forward pass

        # Reorganizar los tag_scores y targets para calcular la pérdida
        # tag_scores: (batch_size, seq_length, num_tags) -> (batch_size * seq_length, num_tags)
        # targets: (batch_size, seq_length) -> (batch_size * seq_length)
        tag_scores = tag_scores.view(-1, tag_scores.size(-1))
        targets = targets.view(-1)

        loss = criterion(tag_scores, targets)
        loss.backward()

        # Recorte de gradientes para evitar explosiones
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    average_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{total_epochs} - Loss: {average_loss:.4f}")

def evaluate(model, dataloader, criterion, device):
    """
    Función de evaluación para calcular la pérdida en el conjunto de validación.

    Args:
        model (nn.Module): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los datos de validación.
        criterion (nn.Module): Función de pérdida.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        float: Pérdida promedio en el conjunto de validación.
    """
    model.eval()
    epoch_loss = 0.0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            tag_scores = model(inputs)
            tag_scores = tag_scores.view(-1, tag_scores.size(-1))
            targets = targets.view(-1)

            loss = criterion(tag_scores, targets)
            epoch_loss += loss.item()

    average_loss = epoch_loss / len(dataloader)
    return average_loss

def save_model(model, optimizer, epoch, loss, path):
    """
    Guarda el estado del modelo y del optimizador.

    Args:
        model (nn.Module): Modelo a guardar.
        optimizer (torch.optim.Optimizer): Optimizador a guardar.
        epoch (int): Número de la época actual.
        loss (float): Pérdida actual.
        path (str): Ruta donde se guardará el modelo.
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path}")

def load_model(model, optimizer, path, device):
    """
    Carga el estado del modelo y del optimizador desde un archivo.

    Args:
        model (nn.Module): Modelo a cargar.
        optimizer (torch.optim.Optimizer): Optimizador a cargar.
        path (str): Ruta desde donde se cargará el modelo.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        int: Época desde la cual se reanudó el entrenamiento.
        float: Pérdida en la última época registrada.
    """
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

def main():
    # Parámetros
    vocab_size = 10000    # Tamaño del vocabulario
    embed_size = 128      # Dimensionalidad de los embeddings
    hidden_size = 256     # Dimensionalidad del estado oculto
    num_tags = 10         # Número de etiquetas POS
    num_layers = 2        # Número de capas de la LSTM
    dropout = 0.5         # Tasa de dropout
    sequence_length = 5   # Longitud de la secuencia
    batch_size = 32       # Tamaño del lote
    num_epochs = 5        # Número de épocas
    learning_rate = 0.001 # Tasa de aprendizaje
    save_path = 'pos_tagger_rnn.pth'  # Ruta para guardar el modelo

    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    # Inicializar el modelo
    model = POSTaggerRNN(vocab_size, embed_size, hidden_size, num_tags, num_layers, dropout).to(device)

    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Crear datasets y dataloaders
    train_dataset = DummyPOSTaggingDataset(num_samples=10000, seq_length=sequence_length, 
                                          vocab_size=vocab_size, num_tags=num_tags)
    val_dataset = DummyPOSTaggingDataset(num_samples=2000, seq_length=sequence_length, 
                                        vocab_size=vocab_size, num_tags=num_tags)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(save_path):
        start_epoch, _ = load_model(model, optimizer, save_path, device)

    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        train(model, train_loader, criterion, optimizer, device, epoch, num_epochs)
        val_loss = evaluate(model, val_loader, criterion, device)
        print(f"Epoca {epoch+1}/{num_epochs} - Validacion cruzada: {val_loss:.4f}")

        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch + 1, val_loss, save_path)

if __name__ == "__main__":
    main()


**Observaciones**

- Actualmente, el código utiliza datos generados aleatoriamente para fines de demostración. Para una aplicación real, deberías reemplazar DummyPOSTaggingDataset con un dataset que contenga secuencias de texto tokenizadas y sus correspondientes etiquetas POS.Puedes utilizar bibliotecas como torchtext para manejar datasets de texto reales.
- Si trabajas con secuencias más largas, considera ajustar el sequence_length y manejar eficientemente la memoria. Técnicas como truncated backpropagation through time (BPTT) pueden ser útiles para evitar problemas de memoria y mejorar el entrenamiento en secuencias largas.
- La capa de dropout en la LSTM ayuda a prevenir el sobreajuste. Puedes ajustar la tasa de dropout según las necesidades. Además del dropout, puedes implementar otras técnicas de regularización si es necesario.
- Experimenta con diferentes tasas de aprendizaje, tamaños de lote, dimensiones de embeddings y tamaños ocultos para encontrar la mejor configuración para tu tarea específica.
- Considera utilizar variantes más avanzadas de RNN, como GRU (nn.GRU), que también manejan bien las dependencias a largo plazo y pueden ser más eficientes computacionalmente que LSTM en ciertos casos.
- Además de la pérdida, puedes implementar métricas como accuracy para monitorear el rendimiento del modelo durante el entrenamiento y la evaluación. Esto proporciona una visión más completa del comportamiento del modelo.

### Clasificación de secuencias utilizando una RNN + FNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os

class RNNSequenceClassifier(nn.Module):
    """
    Modelo de Clasificación de Secuencias basado en una Red Neuronal Recurrente (RNN) y una Red Neuronal Feedforward (FFN).

    Este modelo utiliza una capa de embeddings para convertir tokens de entrada en vectores densos,
    una capa LSTM para procesar secuencias de embeddings y una capa completamente conectada para
    predecir la clase de la secuencia.

    Args:
        vocab_size (int): Tamaño del vocabulario (número de tokens únicos).
        embed_size (int): Dimensionalidad de los embeddings.
        hidden_size (int): Dimensionalidad del estado oculto de la LSTM.
        num_classes (int): Número de clases de salida para la clasificación.
        num_layers (int, opcional): Número de capas de la LSTM. Por defecto es 1.
        dropout (float, opcional): Tasa de dropout entre capas de la LSTM. Por defecto es 0.0.
    """
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes, num_layers=1, dropout=0.0):
        super(RNNSequenceClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)  # Capa de embeddings
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0.0)  # Capa LSTM
        self.fc = nn.Linear(hidden_size, num_classes)  # Capa completamente conectada

        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de las capas lineales y de embeddings usando la inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x):
        """
        Realiza un pase hacia adelante del modelo RNN + FFN para clasificación de secuencias.

        Args:
            x (Tensor): Secuencia de entrada, forma (batch_size, seq_length).

        Returns:
            logits (Tensor): Puntuaciones para cada clase, forma (batch_size, num_classes).
        """
        embeddings = self.embedding(x)  # (batch_size, seq_length, embed_size)
        lstm_out, (h_n, c_n) = self.lstm(embeddings)  # lstm_out: (batch_size, seq_length, hidden_size)
        # Usar el último estado oculto para la clasificación
        final_hidden_state = h_n[-1]  # (batch_size, hidden_size)
        logits = self.fc(final_hidden_state)  # (batch_size, num_classes)
        return logits

class DummySequenceClassificationDataset(Dataset):
    """
    Dataset de ejemplo para Clasificación de Secuencias que genera secuencias aleatorias de tokens y etiquetas.

    Este dataset genera pares de secuencias de entrada y etiquetas de manera aleatoria para fines de demostración.
    """
    def __init__(self, num_samples, seq_length, vocab_size, num_classes):
        """
        Inicializa el dataset.

        Args:
            num_samples (int): Número de muestras en el dataset.
            seq_length (int): Longitud de cada secuencia.
            vocab_size (int): Tamaño del vocabulario.
            num_classes (int): Número de clases de salida.
        """
        super(DummySequenceClassificationDataset, self).__init__()
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.vocab_size = vocab_size
        self.num_classes = num_classes

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Secuencia de entrada (tokens)
        x = torch.randint(0, self.vocab_size, (self.seq_length,))
        # Etiqueta de clase (un valor por secuencia)
        y = torch.randint(0, self.num_classes, (1,)).squeeze()  # Scalar
        return x, y

def train(model, dataloader, criterion, optimizer, device, epoch, total_epochs, clip=5.0):
    """
    Función de entrenamiento para una época.

    Args:
        model (nn.Module): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los datos de entrenamiento.
        criterion (nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        device (torch.device): Dispositivo (CPU o GPU).
        epoch (int): Número de la época actual.
        total_epochs (int): Número total de épocas.
        clip (float, opcional): Valor máximo para el recorte de gradientes. Por defecto es 5.0.
    """
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{total_epochs}")

    for batch_idx, (inputs, targets) in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Mover datos al dispositivo

        optimizer.zero_grad()
        logits = model(inputs)  # Forward pass

        loss = criterion(logits, targets)
        loss.backward()

        # Recorte de gradientes para evitar explosiones
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

        # Cálculo de precisión
        _, predicted = torch.max(logits, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

        average_loss = epoch_loss / (batch_idx + 1)
        accuracy = 100 * correct / total
        progress_bar.set_postfix(loss=average_loss, accuracy=accuracy)

    print(f"Epoch {epoch+1}/{total_epochs} - Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%")

def evaluate(model, dataloader, criterion, device):
    """
    Función de evaluación para calcular la pérdida y precisión en el conjunto de validación.

    Args:
        model (nn.Module): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los datos de validación.
        criterion (nn.Module): Función de pérdida.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        tuple: (pérdida promedio, precisión promedio)
    """
    model.eval()
    epoch_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            logits = model(inputs)
            loss = criterion(logits, targets)
            epoch_loss += loss.item()

            _, predicted = torch.max(logits, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

    average_loss = epoch_loss / len(dataloader)
    accuracy = 100 * correct / total
    return average_loss, accuracy

def save_model(model, optimizer, epoch, loss, path):
    """
    Guarda el estado del modelo y del optimizador.

    Args:
        model (nn.Module): Modelo a guardar.
        optimizer (torch.optim.Optimizer): Optimizador a guardar.
        epoch (int): Número de la época actual.
        loss (float): Pérdida actual.
        path (str): Ruta donde se guardará el modelo.
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path}")

def load_model(model, optimizer, path, device):
    """
    Carga el estado del modelo y del optimizador desde un archivo.

    Args:
        model (nn.Module): Modelo a cargar.
        optimizer (torch.optim.Optimizer): Optimizador a cargar.
        path (str): Ruta desde donde se cargará el modelo.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        tuple: (época desde la cual se reanudó el entrenamiento, pérdida en la última época registrada)
    """
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

def main():
    # Parámetros
    vocab_size = 10000    # Tamaño del vocabulario
    embed_size = 128      # Dimensionalidad de los embeddings
    hidden_size = 256     # Dimensionalidad del estado oculto
    num_classes = 5       # Número de clases de salida para la clasificación
    num_layers = 2        # Número de capas de la LSTM
    dropout = 0.5         # Tasa de dropout
    sequence_length = 10  # Longitud de la secuencia
    batch_size = 32       # Tamaño del lote
    num_epochs = 5        # Número de épocas
    learning_rate = 0.001 # Tasa de aprendizaje
    save_path = 'rnn_sequence_classifier.pth'  # Ruta para guardar el modelo

    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    # Inicializar el modelo
    model = RNNSequenceClassifier(vocab_size, embed_size, hidden_size, num_classes, num_layers, dropout).to(device)

    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Crear datasets y dataloaders
    train_dataset = DummySequenceClassificationDataset(num_samples=10000, seq_length=sequence_length, 
                                                      vocab_size=vocab_size, num_classes=num_classes)
    val_dataset = DummySequenceClassificationDataset(num_samples=2000, seq_length=sequence_length, 
                                                    vocab_size=vocab_size, num_classes=num_classes)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(save_path):
        start_epoch, _ = load_model(model, optimizer, save_path, device)

    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        train(model, train_loader, criterion, optimizer, device, epoch, num_epochs)
        val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
        print(f"Epoca {epoch+1}/{num_epochs} - Validación Loss: {val_loss:.4f}, Validación Accuracy: {val_accuracy:.2f}%")

        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch + 1, val_loss, save_path)

if __name__ == "__main__":
    main()


**Observaciones**

- Actualmente, el código utiliza datos generados aleatoriamente para fines de demostración. Para una aplicación real, deberías reemplazar DummySequenceClassificationDataset con un dataset que contenga secuencias de texto tokenizadas y sus correspondientes etiquetas de clase.
- Puedes utilizar bibliotecas como torchtext para manejar datasets de texto reales.
- Si trabajas con secuencias más largas, considera ajustar el sequence_length y manejar eficientemente la memoria. Técnicas como truncated backpropagation through time (BPTT) pueden ser útiles para evitar problemas de memoria y mejorar el entrenamiento en secuencias largas.
- La capa de dropout en la LSTM ayuda a prevenir el sobreajuste. Puedes ajustar la tasa de dropout según las necesidades. Además del dropout, puedes implementar otras técnicas de regularización si es necesario.
- Experimenta con diferentes tasas de aprendizaje, tamaños de lote, dimensiones de embeddings y tamaños ocultos para encontrar la mejor configuración para tu tarea específica.
- Considera utilizar variantes más avanzadas de RNN, como GRU (nn.GRU), que también manejan bien las dependencias a largo plazo y pueden ser más eficientes computacionalmente que LSTM en ciertos casos.
- Además de la precisión, puedes implementar otras métricas como F1-score, precision y recall para una evaluación más completa del rendimiento del modelo.
- Puedes adaptar el código para usar un dataset real. Este ejemplo utiliza el dataset AG News con la ayuda de torchtext.

### Generación autorregresiva con RNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os
import random
import numpy as np

# Semilla para reproducibilidad
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

class RNNTextGenerator(nn.Module):
    """
    Modelo Autoregresivo RNN para Generación de Texto.

    Este modelo utiliza una capa de embeddings para convertir tokens de entrada en vectores densos,
    una capa LSTM para procesar secuencias de embeddings y una capa completamente conectada para
    predecir la siguiente palabra en la secuencia.

    Args:
        vocab_size (int): Tamaño del vocabulario (número de tokens únicos).
        embed_size (int): Dimensionalidad de los embeddings.
        hidden_size (int): Dimensionalidad del estado oculto de la LSTM.
        num_layers (int, opcional): Número de capas de la LSTM. Por defecto es 1.
        dropout (float, opcional): Tasa de dropout entre capas de la LSTM. Por defecto es 0.0.
    """
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.0):
        super(RNNTextGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)  # Capa de embeddings
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0.0)  # Capa LSTM
        self.fc = nn.Linear(hidden_size, vocab_size)  # Capa completamente conectada para predecir la siguiente palabra

        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de las capas lineales y de embeddings usando la inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, hidden=None):
        """
        Realiza un pase hacia adelante del modelo RNN para generación de texto.

        Args:
            x (Tensor): Secuencia de entrada, forma (batch_size, seq_length).
            hidden (tuple, opcional): Estados ocultos iniciales (h_0, c_0). Si es None, se inicializan a cero.

        Returns:
            logits (Tensor): Puntuaciones para cada palabra en el vocabulario, forma (batch_size, seq_length, vocab_size).
            hidden (tuple): Estados ocultos finales (h_n, c_n).
        """
        embeddings = self.embedding(x)  # (batch_size, seq_length, embed_size)
        lstm_out, hidden = self.lstm(embeddings, hidden)  # lstm_out: (batch_size, seq_length, hidden_size)
        logits = self.fc(lstm_out)  # (batch_size, seq_length, vocab_size)
        return logits, hidden

class TextDataset(Dataset):
    """
    Dataset para Generación de Texto que maneja la tokenización y la creación de secuencias.

    Este dataset toma un texto, lo tokeniza y crea secuencias de longitud fija para el entrenamiento del modelo.
    """
    def __init__(self, text, seq_length, word_to_id, unk_token='<unk>'):
        """
        Inicializa el dataset.

        Args:
            text (str): Texto completo para entrenar el modelo.
            seq_length (int): Longitud de cada secuencia de entrada.
            word_to_id (dict): Mapeo de palabras a IDs.
            unk_token (str, opcional): Token para palabras desconocidas. Por defecto es '<unk>'.
        """
        self.seq_length = seq_length
        self.word_to_id = word_to_id
        self.unk_token = unk_token

        # Tokenizar el texto
        self.tokens = text.lower().split()
        self.data = self.create_sequences()

    def create_sequences(self):
        """
        Crea secuencias de entrada y salida a partir de los tokens.

        Returns:
            list: Lista de tuplas (input_sequence, target_word).
        """
        sequences = []
        for i in range(len(self.tokens) - self.seq_length):
            input_seq = self.tokens[i:i + self.seq_length]
            target = self.tokens[i + self.seq_length]
            input_ids = [self.word_to_id.get(word, self.word_to_id[self.unk_token]) for word in input_seq]
            target_id = self.word_to_id.get(target, self.word_to_id[self.unk_token])
            sequences.append((input_ids, target_id))
        return sequences

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_seq, target = self.data[idx]
        return torch.tensor(input_seq, dtype=torch.long), torch.tensor(target, dtype=torch.long)

def train(model, dataloader, criterion, optimizer, device, epoch, total_epochs, clip=5.0):
    """
    Función de entrenamiento para una época.

    Args:
        model (nn.Module): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los datos de entrenamiento.
        criterion (nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        device (torch.device): Dispositivo (CPU o GPU).
        epoch (int): Número de la época actual.
        total_epochs (int): Número total de épocas.
        clip (float, opcional): Valor máximo para el recorte de gradientes. Por defecto es 5.0.
    """
    model.train()
    epoch_loss = 0.0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{total_epochs}")

    for batch_idx, (inputs, targets) in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Mover datos al dispositivo

        optimizer.zero_grad()
        logits, hidden = model(inputs)  # Forward pass

        # Extraer solo las predicciones del último token en la secuencia
        # logits: (batch_size, seq_length, vocab_size) -> (batch_size, vocab_size)
        logits = logits[:, -1, :]

        loss = criterion(logits, targets)
        loss.backward()

        # Recorte de gradientes para evitar explosiones
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
        average_loss = epoch_loss / (batch_idx + 1)
        progress_bar.set_postfix(loss=average_loss)

    print(f"Epoch {epoch+1}/{total_epochs} - Loss: {average_loss:.4f}")

def evaluate(model, dataloader, criterion, device):
    """
    Función de evaluación para calcular la pérdida en el conjunto de validación.

    Args:
        model (nn.Module): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los datos de validación.
        criterion (nn.Module): Función de pérdida.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        float: Pérdida promedio en el conjunto de validación.
    """
    model.eval()
    epoch_loss = 0.0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            logits, hidden = model(inputs)
            logits = logits[:, -1, :]  # (batch_size, vocab_size)

            loss = criterion(logits, targets)
            epoch_loss += loss.item()

    average_loss = epoch_loss / len(dataloader)
    return average_loss

def save_model(model, optimizer, epoch, loss, path):
    """
    Guarda el estado del modelo y del optimizador.

    Args:
        model (nn.Module): Modelo a guardar.
        optimizer (torch.optim.Optimizer): Optimizador a guardar.
        epoch (int): Número de la época actual.
        loss (float): Pérdida actual.
        path (str): Ruta donde se guardará el modelo.
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path}")

def load_model(model, optimizer, path, device):
    """
    Carga el estado del modelo y del optimizador desde un archivo.

    Args:
        model (nn.Module): Modelo a cargar.
        optimizer (torch.optim.Optimizer): Optimizador a cargar.
        path (str): Ruta desde donde se cargará el modelo.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        tuple: (época desde la cual se reanudó el entrenamiento, pérdida en la última época registrada)
    """
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

def generate_text(model, start_token, id_to_word, word_to_id, length=5, device='cpu', temperature=1.0):
    """
    Genera una secuencia de texto de forma autoregresiva.

    Args:
        model (nn.Module): Modelo entrenado para generación de texto.
        start_token (int): ID del token inicial (por ejemplo, <s>).
        id_to_word (dict): Mapeo de IDs a palabras.
        word_to_id (dict): Mapeo de palabras a IDs.
        length (int, opcional): Longitud de la secuencia a generar. Por defecto es 5.
        device (str, opcional): Dispositivo para generar el texto. Por defecto es 'cpu'.
        temperature (float, opcional): Controla la aleatoriedad de la predicción. Por defecto es 1.0.

    Returns:
        list: Lista de palabras generadas.
    """
    model.eval()
    input_word = torch.LongTensor([[start_token]]).to(device)  # Palabra inicial (<s>)
    hidden = None  # Estado oculto inicial
    generated_words = [id_to_word[start_token]]  # Inicializamos con el token de inicio

    with torch.no_grad():
        for _ in range(length):
            logits, hidden = model(input_word, hidden)
            logits = logits[:, -1, :] / temperature
            probs = torch.softmax(logits, dim=-1)
            next_word_id = torch.multinomial(probs, num_samples=1).item()
            generated_words.append(id_to_word.get(next_word_id, '<unk>'))
            input_word = torch.LongTensor([[next_word_id]]).to(device)

    return generated_words

def main():
    # Parámetros
    vocab_size = 10000    # Tamaño del vocabulario
    embed_size = 128      # Dimensionalidad de los embeddings
    hidden_size = 256     # Dimensionalidad del estado oculto
    num_layers = 2        # Número de capas de la LSTM
    dropout = 0.5         # Tasa de dropout
    sequence_length = 5   # Longitud de la secuencia
    batch_size = 64       # Tamaño del lote
    num_epochs = 10       # Número de épocas
    learning_rate = 0.001 # Tasa de aprendizaje
    save_path = 'rnn_text_generator.pth'  # Ruta para guardar el modelo

    # Configuración del dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    # Ejemplo de texto (reemplaza esto con un dataset real para mejores resultados)
    sample_text = "So long and thanks for all the fish. So long and thanks for all the fish."

    # Crear mappings de palabras a IDs y viceversa
    words = sample_text.lower().split()
    unique_words = set(words)
    word_to_id = {word: idx for idx, word in enumerate(unique_words, start=1)}  # Empezar en 1
    word_to_id['<s>'] = 0  # Token de inicio
    word_to_id['<unk>'] = len(word_to_id)  # Token desconocido
    id_to_word = {idx: word for word, idx in word_to_id.items()}

    vocab_size = len(word_to_id)

    # Crear el dataset y el dataloader
    dataset = TextDataset(text=sample_text, seq_length=sequence_length, word_to_id=word_to_id)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Inicializar el modelo
    model = RNNTextGenerator(vocab_size, embed_size, hidden_size, num_layers, dropout).to(device)

    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(save_path):
        start_epoch, _ = load_model(model, optimizer, save_path, device)

    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        train(model, dataloader, criterion, optimizer, device, epoch, num_epochs)
        val_loss = evaluate(model, dataloader, criterion, device)
        print(f"Epoca {epoch+1}/{num_epochs} - Validación Loss: {val_loss:.4f}")

        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch + 1, val_loss, save_path)

    # Generar una secuencia de texto
    start_token = word_to_id['<s>']
    generated_sequence = generate_text(model, start_token, id_to_word, word_to_id, length=10, device=device, temperature=1.0)
    print("Secuencia generada:", " ".join(generated_sequence))

if __name__ == "__main__":
    main()


**Observaciones**

- El sample_text utilizado en el ejemplo es muy pequeño, lo que limita la capacidad del modelo para aprender patrones significativos. Para obtener mejores resultados, considera entrenar el modelo en un corpus de texto más grande y diverso, como los proporcionados por WikiText, BooksCorpus, o datasets de Hugging Face.
- Actualmente, el modelo trunca las secuencias que exceden seq_length. Para manejar secuencias de longitud variable de manera más efectiva, podrías considerar el padding y el uso de pack_padded_sequence y pad_packed_sequence de PyTorch. Esto es especialmente útil si deseas que el modelo pueda manejar entradas de diferentes longitudes sin perder información.
- Además del dropout ya implementado, considera añadir técnicas de regularización adicionales, como la normalización de pesos (Weight Normalization) o la utilización de capas adicionales, para mejorar la capacidad de generalización del modelo y prevenir el sobreajuste.
- Experimenta con diferentes tasas de aprendizaje, tamaños de lote, dimensiones de embeddings y tamaños ocultos para encontrar la mejor configuración para tu tarea específica. Puedes utilizar herramientas como Optuna o Ray Tune para automatizar la búsqueda de hiperparámetros.
- Considera utilizar variantes más avanzadas de RNN, como GRU (nn.GRU), que pueden ser más eficientes computacionalmente que LSTM en ciertos casos. Además, podrías explorar arquitecturas más modernas como Transformers, que han demostrado un rendimiento superior en tareas de generación de texto.
- Además de la pérdida (loss), considera evaluar el modelo utilizando métricas adicionales como la perplejidad (perplexity), que es una medida comúnmente utilizada en modelos de lenguaje para evaluar la capacidad de predicción.
- Para mejorar la calidad del texto generado, puedes implementar técnicas de muestreo más avanzadas, como top-k sampling o nucleus sampling (top-p sampling), que permiten un mejor control sobre la diversidad y coherencia del texto generado.
- Se puede adaptar del código para entrenar el modelo utilizando el dataset WikiText-2 con la ayuda de torchtext. Este ejemplo reemplazaria el texto de muestra con un corpus más grande y maneja la construcción del vocabulario de manera más robusta.

### Redes recurrentes apiladas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os
import random
import numpy as np

# Establecer semillas para reproducibilidad
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Verificar dispositivo (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

class StackedLSTM(nn.Module):
    """
    Modelo LSTM apilado para generación de texto.
    
    Args:
        vocab_size (int): Tamaño del vocabulario.
        embed_size (int): Dimensionalidad de los embeddings.
        hidden_size (int): Dimensionalidad del estado oculto de la LSTM.
        num_layers (int): Número de capas de LSTM apiladas.
        dropout (float): Tasa de dropout entre capas de LSTM.
    """
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, dropout=0.5):
        super(StackedLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers, 
                            batch_first=True, dropout=dropout if num_layers >1 else 0.0)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
        self.init_weights()
    
    def init_weights(self):
        """
        Inicializa los pesos de las capas de embeddings, LSTM y fully connected usando la inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)
    
    def forward(self, x, hidden=None):
        """
        Pase hacia adelante del modelo LSTM apilado para generación de texto.
        
        Args:
            x (Tensor): Secuencia de entrada, forma (batch_size, seq_length).
            hidden (tuple, opcional): Estados ocultos iniciales (h_0, c_0). Si es None, se inicializan a cero.
        
        Returns:
            logits (Tensor): Puntuaciones para cada palabra en el vocabulario, forma (batch_size, seq_length, vocab_size).
            hidden (tuple): Estados ocultos finales (h_n, c_n).
        """
        embed = self.embedding(x)  # (batch_size, seq_length, embed_size)
        lstm_out, hidden = self.lstm(embed, hidden)  # lstm_out: (batch_size, seq_length, hidden_size)
        logits = self.fc(lstm_out)  # (batch_size, seq_length, vocab_size)
        return logits, hidden

class LanguageModelDataset(Dataset):
    """
    Dataset para modelado de lenguaje que maneja la tokenización y creación de secuencias.
    
    Cada muestra consiste en una secuencia de tokens de entrada y el token objetivo que sigue a la secuencia.
    
    Args:
        text (str): Texto completo para entrenar el modelo.
        seq_length (int): Longitud de las secuencias de entrada.
        word_to_id (dict): Mapeo de palabras a IDs.
        unk_token (str): Token para palabras desconocidas.
    """
    def __init__(self, text, seq_length, word_to_id, unk_token='<unk>'):
        self.seq_length = seq_length
        self.word_to_id = word_to_id
        self.unk_token = unk_token
        self.tokens = text.lower().split()
        self.data = self.create_sequences()
    
    def create_sequences(self):
        """
        Crea secuencias de entrada y salida a partir de los tokens.
        
        Returns:
            list: Lista de tuplas (input_sequence, target_word).
        """
        sequences = []
        for i in range(len(self.tokens) - self.seq_length):
            seq = self.tokens[i:i + self.seq_length]
            target = self.tokens[i + self.seq_length]
            seq_ids = [self.word_to_id.get(word, self.word_to_id[self.unk_token]) for word in seq]
            target_id = self.word_to_id.get(target, self.word_to_id[self.unk_token])
            sequences.append((seq_ids, target_id))
        return sequences
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        seq, target = self.data[idx]
        return torch.tensor(seq, dtype=torch.long), torch.tensor(target, dtype=torch.long)

def train_epoch(model, dataloader, criterion, optimizer, device, clip=5.0):
    """
    Entrena el modelo durante una época.
    
    Args:
        model (nn.Module): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los datos de entrenamiento.
        criterion (nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        device (torch.device): Dispositivo (CPU o GPU).
        clip (float): Valor máximo para el recorte de gradientes.
    
    Returns:
        float: Pérdida promedio para la época.
    """
    model.train()
    total_loss = 0.0
    progress = tqdm(dataloader, desc="Entrenando", leave=False)
    
    for inputs, targets in progress:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        logits, _ = model(inputs)  # logits: (batch_size, seq_length, vocab_size)
        # Nos interesan las predicciones del último token de la secuencia
        logits = logits[:, -1, :]  # (batch_size, vocab_size)
        loss = criterion(logits, targets)
        loss.backward()
        
        # Recorte de gradientes
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        total_loss += loss.item()
        progress.set_postfix(loss=loss.item())
    
    average_loss = total_loss / len(dataloader)
    return average_loss

def evaluate_epoch(model, dataloader, criterion, device):
    """
    Evalúa el modelo durante una época.
    
    Args:
        model (nn.Module): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los datos de evaluación.
        criterion (nn.Module): Función de pérdida.
        device (torch.device): Dispositivo (CPU o GPU).
    
    Returns:
        float: Pérdida promedio para la evaluación.
    """
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        progress = tqdm(dataloader, desc="Evaluando", leave=False)
        for inputs, targets in progress:
            inputs, targets = inputs.to(device), targets.to(device)
            logits, _ = model(inputs)
            logits = logits[:, -1, :]  # (batch_size, vocab_size)
            loss = criterion(logits, targets)
            total_loss += loss.item()
            progress.set_postfix(loss=loss.item())
    
    average_loss = total_loss / len(dataloader)
    return average_loss

def save_checkpoint(model, optimizer, epoch, loss, path):
    """
    Guarda el estado del modelo y del optimizador.
    
    Args:
        model (nn.Module): Modelo a guardar.
        optimizer (torch.optim.Optimizer): Optimizador a guardar.
        epoch (int): Época actual.
        loss (float): Pérdida actual.
        path (str): Ruta donde se guardará el modelo.
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path} (Época {epoch}, Pérdida {loss:.4f})")

def load_checkpoint(model, optimizer, path, device):
    """
    Carga el estado del modelo y del optimizador desde un archivo.
    
    Args:
        model (nn.Module): Modelo a cargar.
        optimizer (torch.optim.Optimizer): Optimizador a cargar.
        path (str): Ruta desde donde se cargará el modelo.
        device (torch.device): Dispositivo (CPU o GPU).
    
    Returns:
        tuple: (época, pérdida) cargados desde el checkpoint.
    """
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss:.4f})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

def generate_text(model, start_token, id_to_word, word_to_id, length=20, device='cpu', temperature=1.0):
    """
    Genera una secuencia de texto de forma autoregresiva.
    
    Args:
        model (nn.Module): Modelo entrenado para generación de texto.
        start_token (int): ID del token inicial (por ejemplo, <s>).
        id_to_word (dict): Mapeo de IDs a palabras.
        word_to_id (dict): Mapeo de palabras a IDs.
        length (int, opcional): Longitud de la secuencia a generar. Por defecto es 20.
        device (torch.device, opcional): Dispositivo para generar el texto. Por defecto es 'cpu'.
        temperature (float, opcional): Controla la aleatoriedad de la predicción. Por defecto es 1.0.
    
    Returns:
        list: Lista de palabras generadas.
    """
    model.eval()
    generated = [id_to_word[start_token]]
    input_seq = torch.tensor([[start_token]], dtype=torch.long).to(device)
    hidden = None
    
    with torch.no_grad():
        for _ in range(length):
            logits, hidden = model(input_seq, hidden)
            logits = logits[:, -1, :] / temperature
            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            generated.append(id_to_word.get(next_token, '<unk>'))
            input_seq = torch.tensor([[next_token]], dtype=torch.long).to(device)
    
    return generated

def main():
    # Parámetros
    vocab_size = 10000    # Tamaño del vocabulario (se ajustará basado en los datos)
    embed_size = 128      # Dimensionalidad de los embeddings
    hidden_size = 256     # Dimensionalidad del estado oculto
    num_layers = 3        # Número de capas de la LSTM
    dropout = 0.5         # Tasa de dropout
    sequence_length = 10  # Longitud de la secuencia
    batch_size = 32       # Tamaño del lote
    num_epochs = 5        # Número de épocas
    learning_rate = 0.001 # Tasa de aprendizaje
    checkpoint_path = 'stacked_lstm_checkpoint.pth'  # Ruta para guardar el modelo
    
    # Ejemplo de texto (reemplaza esto con un dataset real para mejores resultados)
    sample_text = (
        "So long and thanks for all the fish. "
        "So long and thanks for all the fish. "
        "So long and thanks for all the fish."
    )
    
    # Crear mappings de palabras a IDs y viceversa
    words = sample_text.lower().split()
    unique_words = set(words)
    word_to_id = {word: idx for idx, word in enumerate(unique_words, start=1)}  # Empezar en 1
    word_to_id['<s>'] = 0  # Token de inicio
    word_to_id['<unk>'] = len(word_to_id)  # Token desconocido
    id_to_word = {idx: word for word, idx in word_to_id.items()}
    
    vocab_size = len(word_to_id)
    
    # Crear el dataset y el dataloader
    dataset = LanguageModelDataset(text=sample_text, seq_length=sequence_length, word_to_id=word_to_id)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Inicializar el modelo, función de pérdida y optimizador
    model = StackedLSTM(vocab_size, embed_size, hidden_size, num_layers, dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(checkpoint_path):
        start_epoch, _ = load_checkpoint(model, optimizer, checkpoint_path, device)
    
    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        print(f"\nEpoca {epoch+1}/{num_epochs}")
        train_loss = train_epoch(model, dataloader, criterion, optimizer, device)
        print(f"Entrenamiento - Pérdida: {train_loss:.4f}")
        
        # Evaluar en el mismo conjunto de datos (para demostración; usa un conjunto de validación separado en la práctica)
        val_loss = evaluate_epoch(model, dataloader, criterion, device)
        print(f"Validación - Pérdida: {val_loss:.4f}")
        
        # Guardar el checkpoint
        save_checkpoint(model, optimizer, epoch+1, val_loss, checkpoint_path)
    
    # Generar una secuencia de texto
    start_word = '<s>'
    start_token = word_to_id.get(start_word, word_to_id['<unk>'])
    generated = generate_text(model, start_token, id_to_word, word_to_id, length=10, device=device, temperature=1.0)
    print("\nSecuencia generada:", " ".join(generated))

if __name__ == "__main__":
    main()


**Observación**

Para obtener resultados más significativos, es recomendable entrenar el modelo en un corpus de texto más grande y variado. Puedes utilizar datasets como WikiText-2, Penn Treebank, o cualquier otro corpus de texto que prefieras.

### Redes recurrentes bidireccionales para clasificación de secuencia

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os
import random
import numpy as np

# Establecer semillas para reproducibilidad
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Verificar dispositivo (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

class BidirectionalLSTMClassifier(nn.Module):
    """
    Modelo de clasificación de secuencias utilizando LSTM bidireccional.
    
    Args:
        vocab_size (int): Tamaño del vocabulario.
        embed_size (int): Dimensionalidad de los embeddings.
        hidden_size (int): Dimensionalidad del estado oculto de la LSTM.
        num_layers (int): Número de capas de LSTM apiladas.
        output_size (int): Número de clases para la clasificación.
        dropout (float): Tasa de dropout entre capas de LSTM.
    """
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(BidirectionalLSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers, 
                            bidirectional=True, batch_first=True, dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # *2 por bidireccionalidad
        self.dropout = nn.Dropout(dropout)
        
        self.init_weights()
    
    def init_weights(self):
        """
        Inicializa los pesos de las capas de embeddings, LSTM y fully connected usando la inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)
    
    def forward(self, x):
        """
        Pase hacia adelante del modelo LSTM bidireccional para clasificación de secuencias.
        
        Args:
            x (Tensor): Secuencia de entrada, forma (batch_size, seq_length).
        
        Returns:
            logits (Tensor): Puntuaciones para cada clase, forma (batch_size, output_size).
        """
        embed = self.embedding(x)  # (batch_size, seq_length, embed_size)
        lstm_out, (hidden, cell) = self.lstm(embed)  # lstm_out: (batch_size, seq_length, hidden_size*2)
        
        # Concatenar los estados ocultos finales de ambas direcciones
        hidden_forward = hidden[-2, :, :]  # Última capa, dirección forward
        hidden_backward = hidden[-1, :, :]  # Última capa, dirección backward
        hidden_cat = torch.cat((hidden_forward, hidden_backward), dim=1)  # (batch_size, hidden_size*2)
        
        out = self.dropout(hidden_cat)
        logits = self.fc(out)  # (batch_size, output_size)
        return logits

class SequenceClassificationDataset(Dataset):
    """
    Dataset para clasificación de secuencias que maneja la creación de secuencias de entrada y etiquetas.
    
    Cada muestra consiste en una secuencia de tokens de entrada y una etiqueta de clase.
    
    Args:
        sequences (list of list of int): Lista de secuencias de tokens.
        labels (list of int): Lista de etiquetas de clase correspondientes a cada secuencia.
    """
    def __init__(self, sequences, labels):
        assert len(sequences) == len(labels), "Las secuencias y las etiquetas deben tener la misma longitud."
        self.sequences = sequences
        self.labels = labels
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

def train_epoch(model, dataloader, criterion, optimizer, device, clip=5.0):
    """
    Entrena el modelo durante una época.
    
    Args:
        model (nn.Module): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los datos de entrenamiento.
        criterion (nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        device (torch.device): Dispositivo (CPU o GPU).
        clip (float): Valor máximo para el recorte de gradientes.
    
    Returns:
        float: Pérdida promedio para la época.
        float: Exactitud promedio para la época.
    """
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    progress = tqdm(dataloader, desc="Entrenando", leave=False)
    
    for inputs, targets in progress:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        logits = model(inputs)  # (batch_size, output_size)
        loss = criterion(logits, targets)
        loss.backward()
        
        # Recorte de gradientes
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        total_loss += loss.item()
        
        # Cálculo de exactitud
        _, predicted = torch.max(logits, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)
        
        avg_loss = total_loss / (len(dataloader))
        accuracy = 100.0 * correct / total
        progress.set_postfix(loss=avg_loss, accuracy=f"{accuracy:.2f}%")
    
    average_loss = total_loss / len(dataloader)
    average_accuracy = 100.0 * correct / total
    return average_loss, average_accuracy

def evaluate_epoch(model, dataloader, criterion, device):
    """
    Evalúa el modelo durante una época.
    
    Args:
        model (nn.Module): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los datos de evaluación.
        criterion (nn.Module): Función de pérdida.
        device (torch.device): Dispositivo (CPU o GPU).
    
    Returns:
        float: Pérdida promedio para la evaluación.
        float: Exactitud promedio para la evaluación.
    """
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        progress = tqdm(dataloader, desc="Evaluando", leave=False)
        for inputs, targets in progress:
            inputs, targets = inputs.to(device), targets.to(device)
            logits = model(inputs)  # (batch_size, output_size)
            loss = criterion(logits, targets)
            total_loss += loss.item()
            
            # Cálculo de exactitud
            _, predicted = torch.max(logits, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
            
            avg_loss = total_loss / len(dataloader)
            accuracy = 100.0 * correct / total
            progress.set_postfix(loss=avg_loss, accuracy=f"{accuracy:.2f}%")
    
    average_loss = total_loss / len(dataloader)
    average_accuracy = 100.0 * correct / total
    return average_loss, average_accuracy

def save_checkpoint(model, optimizer, epoch, loss, path):
    """
    Guarda el estado del modelo y del optimizador.
    
    Args:
        model (nn.Module): Modelo a guardar.
        optimizer (torch.optim.Optimizer): Optimizador a guardar.
        epoch (int): Época actual.
        loss (float): Pérdida actual.
        path (str): Ruta donde se guardará el modelo.
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path} (Época {epoch}, Pérdida {loss:.4f})")

def load_checkpoint(model, optimizer, path, device):
    """
    Carga el estado del modelo y del optimizador desde un archivo.
    
    Args:
        model (nn.Module): Modelo a cargar.
        optimizer (torch.optim.Optimizer): Optimizador a cargar.
        path (str): Ruta desde donde se cargará el modelo.
        device (torch.device): Dispositivo (CPU o GPU).
    
    Returns:
        tuple: (época, pérdida) cargados desde el checkpoint.
    """
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss:.4f})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

def generate_text(model, start_token, id_to_word, word_to_id, length=20, device='cpu', temperature=1.0):
    """
    Genera una secuencia de texto de forma autoregresiva.
    
    Args:
        model (nn.Module): Modelo entrenado para generación de texto.
        start_token (int): ID del token inicial (por ejemplo, <s>).
        id_to_word (dict): Mapeo de IDs a palabras.
        word_to_id (dict): Mapeo de palabras a IDs.
        length (int, opcional): Longitud de la secuencia a generar. Por defecto es 20.
        device (torch.device, opcional): Dispositivo para generar el texto. Por defecto es 'cpu'.
        temperature (float, opcional): Controla la aleatoriedad de la predicción. Por defecto es 1.0.
    
    Returns:
        list: Lista de palabras generadas.
    """
    model.eval()
    generated = [id_to_word[start_token]]
    input_seq = torch.tensor([[start_token]], dtype=torch.long).to(device)
    hidden = None
    
    with torch.no_grad():
        for _ in range(length):
            logits = model(input_seq)  # (batch_size=1, output_size)
            logits = logits / temperature
            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            generated.append(id_to_word.get(next_token, '<unk>'))
            input_seq = torch.tensor([[next_token]], dtype=torch.long).to(device)
    
    return generated

def main():
    # Parámetros
    vocab_size = 10000    # Tamaño del vocabulario
    embed_size = 128      # Dimensionalidad de los embeddings
    hidden_size = 256     # Dimensionalidad del estado oculto
    num_layers = 3        # Número de capas de LSTM apiladas
    output_size = 5       # Número de clases para clasificación
    dropout = 0.5         # Tasa de dropout
    sequence_length = 10  # Longitud de la secuencia
    batch_size = 32       # Tamaño del lote
    num_epochs = 10       # Número de épocas
    learning_rate = 0.001 # Tasa de aprendizaje
    checkpoint_path = 'bidirectional_lstm_classifier.pth'  # Ruta para guardar el modelo
    
    # Ejemplo de texto (reemplaza esto con un dataset real para mejores resultados)
    sample_text = (
        "Este es un ejemplo de texto para la clasificación de secuencias. "
        "Las redes neuronales recurrentes bidireccionales pueden capturar información contextual "
        "desde ambas direcciones de una secuencia. Esto es útil para tareas como la clasificación "
        "de texto, donde el significado de una palabra puede depender de las palabras que la preceden "
        "y la siguen."
    )
    
    # Preprocesamiento: Tokenización y creación de vocabulario
    words = sample_text.lower().split()
    unique_words = set(words)
    word_to_id = {word: idx for idx, word in enumerate(unique_words, start=1)}  # Empezar en 1
    word_to_id['<s>'] = 0  # Token de inicio
    word_to_id['<unk>'] = len(word_to_id)  # Token desconocido
    id_to_word = {idx: word for word, idx in word_to_id.items()}
    
    vocab_size = len(word_to_id)
    
    # Generar datos de ejemplo (para demostración; usa datos reales en práctica)
    num_samples = 1000
    sequences = []
    labels = []
    for _ in range(num_samples):
        seq = random.randint(0, len(words) - sequence_length - 1)
        input_seq = words[seq:seq + sequence_length]
        target = random.randint(0, output_size - 1)  # Etiquetas aleatorias
        input_ids = [word_to_id.get(word, word_to_id['<unk>']) for word in input_seq]
        sequences.append(input_ids)
        labels.append(target)
    
    # Crear el dataset y el dataloader
    dataset = SequenceClassificationDataset(sequences, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Inicializar el modelo, función de pérdida y optimizador
    model = BidirectionalLSTMClassifier(vocab_size, embed_size, hidden_size, num_layers, output_size, dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(checkpoint_path):
        start_epoch, _ = load_checkpoint(model, optimizer, checkpoint_path, device)
    
    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        print(f"\nEpoca {epoch+1}/{num_epochs}")
        train_loss, train_acc = train_epoch(model, dataloader, criterion, optimizer, device)
        print(f"Entrenamiento - Pérdida: {train_loss:.4f}, Exactitud: {train_acc:.2f}%")
        
        # Evaluar en el mismo conjunto de datos (para demostración; usa un conjunto de validación separado en la práctica)
        val_loss, val_acc = evaluate_epoch(model, dataloader, criterion, device)
        print(f"Validación - Pérdida: {val_loss:.4f}, Exactitud: {val_acc:.2f}%")
        
        # Guardar el checkpoint
        save_checkpoint(model, optimizer, epoch+1, val_loss, checkpoint_path)
    
    # Generar una secuencia de texto (opcional; depende de la tarea)
    # Aquí, la generación de texto no es directamente aplicable ya que el modelo está diseñado para clasificación.
    # Sin embargo, si adaptas el modelo para generación de texto, puedes usar una función similar a la anterior.

if __name__ == "__main__":
    main()


**Consideraciones adicionales**

- En el ejemplo proporcionado, se generan datos aleatorios para la demostración. Para una aplicación real, reemplaza esta parte con un dataset real de clasificación de secuencias. Puedes usar datasets como IMDB para análisis de sentimientos, AG News o cualquier otro dataset relevante.
- Actualmente, tanto el entrenamiento como la evaluación se realizan en el mismo conjunto de datos generado aleatoriamente. En una práctica real, deberías separar tus datos en conjuntos de entrenamiento y validación para evaluar el rendimiento del modelo de manera más precisa.
- Si trabajas con secuencias de longitud variable, considera usar técnicas de padding y pack_padded_sequence y pad_packed_sequence de PyTorch para manejar eficientemente las secuencias de diferentes longitudes.
- Además del dropout, puedes explorar otras técnicas de regularización como la normalización de pesos (Weight Normalization) o el uso de capas adicionales para mejorar la capacidad de generalización del modelo.
- Experimenta con diferentes tasas de aprendizaje, tamaños de lote, dimensiones de embeddings y tamaños ocultos para encontrar la mejor configuración para tu tarea específica. Herramientas como Optuna o Ray Tune pueden ayudarte a automatizar la búsqueda de hiperparámetros.
- Si adaptas el modelo para tareas generativas, considera técnicas como top-k sampling o nucleus sampling (top-p sampling) para controlar mejor la diversidad y coherencia del texto generado.

### LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os
import random
import numpy as np

# Establecer semillas para reproducibilidad
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Verificar dispositivo (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

class LSTMCell(nn.Module):
    """
    Implementación personalizada de una celda LSTM.

    Args:
        input_size (int): Tamaño del vector de entrada (dimensionalidad de x_t).
        hidden_size (int): Tamaño del estado oculto (dimensionalidad de h_t).
    """
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Capa lineal para los gates y el candidato de la celda
        self.fc = nn.Linear(input_size + hidden_size, 4 * hidden_size)

        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de la capa lineal usando la inicialización Xavier uniforme
        y los sesgos a cero.
        """
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x_t, h_t_minus_1, c_t_minus_1):
        """
        Realiza un pase hacia adelante de la celda LSTM.

        Args:
            x_t (Tensor): Entrada en el tiempo t, forma (batch_size, input_size).
            h_t_minus_1 (Tensor): Estado oculto anterior, forma (batch_size, hidden_size).
            c_t_minus_1 (Tensor): Estado de la celda anterior, forma (batch_size, hidden_size).

        Returns:
            h_t (Tensor): Nuevo estado oculto, forma (batch_size, hidden_size).
            c_t (Tensor): Nuevo estado de la celda, forma (batch_size, hidden_size).
        """
        # Concatenar la entrada y el estado oculto anterior
        combined = torch.cat((x_t, h_t_minus_1), dim=1)  # (batch_size, input_size + hidden_size)

        # Aplicar la capa lineal
        gates = self.fc(combined)  # (batch_size, 4 * hidden_size)
        i_gate, f_gate, g_gate, o_gate = gates.chunk(4, dim=1)

        # Aplicar funciones de activación
        i_t = torch.sigmoid(i_gate)
        f_t = torch.sigmoid(f_gate)
        g_t = torch.tanh(g_gate)
        o_t = torch.sigmoid(o_gate)

        # Actualizar el estado de la celda
        c_t = f_t * c_t_minus_1 + i_t * g_t

        # Actualizar el estado oculto
        h_t = o_t * torch.tanh(c_t)

        return h_t, c_t

class SequenceDataset(Dataset):
    """
    Dataset personalizado para manejar secuencias y sus etiquetas.

    Args:
        sequences (list of list of float): Lista de secuencias de vectores de entrada.
        labels (list of int): Lista de etiquetas correspondientes a cada secuencia.
    """
    def __init__(self, sequences, labels):
        assert len(sequences) == len(labels), "Las secuencias y las etiquetas deben tener la misma longitud."
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        # Secuencia: (seq_length, input_size) -> torch.float32
        return torch.tensor(self.sequences[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

class LSTMModel(nn.Module):
    """
    Modelo completo que utiliza una celda LSTM personalizada para procesar secuencias y realizar clasificación.

    Args:
        input_size (int): Tamaño del vector de entrada.
        hidden_size (int): Tamaño del estado oculto de la LSTM.
        output_size (int): Tamaño de la salida (número de clases).
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm_cell = LSTMCell(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)  # Mapea el estado oculto a logits de clases
        self.init_weights()

    def init_weights(self):
        """
        Inicializa los pesos de la capa fully connected usando la inicialización Xavier uniforme
        y los sesgos a cero.
        """
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, h_0=None, c_0=None):
        """
        Realiza un pase hacia adelante del modelo LSTM.

        Args:
            x (Tensor): Entrada, forma (batch_size, seq_length, input_size).
            h_0 (Tensor, opcional): Estado oculto inicial, forma (batch_size, hidden_size).
            c_0 (Tensor, opcional): Estado de la celda inicial, forma (batch_size, hidden_size).

        Returns:
            logits (Tensor): Puntuaciones para cada clase, forma (batch_size, output_size).
        """
        batch_size, seq_length, _ = x.size()
        if h_0 is None:
            h_t = torch.zeros(batch_size, self.hidden_size, device=x.device)
        else:
            h_t = h_0
        if c_0 is None:
            c_t = torch.zeros(batch_size, self.hidden_size, device=x.device)
        else:
            c_t = c_0

        for t in range(seq_length):
            x_t = x[:, t, :]  # (batch_size, input_size)
            h_t, c_t = self.lstm_cell(x_t, h_t, c_t)

        logits = self.fc(h_t)  # (batch_size, output_size)
        return logits

def train(model, dataloader, criterion, optimizer, device, epoch, total_epochs, clip=5.0):
    """
    Función de entrenamiento para una época.

    Args:
        model (nn.Module): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los datos de entrenamiento.
        criterion (nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        device (torch.device): Dispositivo (CPU o GPU).
        epoch (int): Número de la época actual.
        total_epochs (int): Número total de épocas.
        clip (float, opcional): Valor máximo para el recorte de gradientes. Por defecto es 5.0.
    """
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{total_epochs}")

    for batch_idx, (inputs, targets) in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)  # targets: [batch_size, 1]

        # Inicializar los estados ocultos y de la celda
        h_t = torch.zeros(inputs.size(0), model.hidden_size, device=device)
        c_t = torch.zeros(inputs.size(0), model.hidden_size, device=device)

        optimizer.zero_grad()

        # Procesar cada paso temporal
        for t in range(inputs.size(1)):
            x_t = inputs[:, t, :]  # (batch_size, input_size)
            h_t, c_t = model.lstm_cell(x_t, h_t, c_t)

        # Calcular la salida
        logits = model.fc(h_t)  # (batch_size, output_size)

        # Calcular la pérdida
        loss = criterion(logits, targets)

        loss.backward()

        # Recorte de gradientes para evitar explosiones
        torch.nn.utils.clip_grad_norm_(model.lstm_cell.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

        # Cálculo de exactitud
        predicted = torch.round(torch.sigmoid(logits))  # Para BCEWithLogitsLoss
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

        # Actualizar la barra de progreso
        average_loss = epoch_loss / (batch_idx + 1)
        accuracy = 100.0 * correct / total
        progress_bar.set_postfix(loss=average_loss, accuracy=f"{accuracy:.2f}%")

    print(f"Epoch {epoch+1}/{total_epochs} - Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%")

def evaluate(model, dataloader, criterion, device):
    """
    Función de evaluación para calcular la pérdida en el conjunto de validación.

    Args:
        model (nn.Module): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los datos de validación.
        criterion (nn.Module): Función de pérdida.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        float: Pérdida promedio en el conjunto de validación.
        float: Exactitud promedio en el conjunto de validación.
    """
    model.eval()
    epoch_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(dataloader, desc="Evaluando", leave=False)
    with torch.no_grad():
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)

            # Inicializar los estados ocultos y de la celda
            h_t = torch.zeros(inputs.size(0), model.hidden_size, device=device)
            c_t = torch.zeros(inputs.size(0), model.hidden_size, device=device)

            # Procesar cada paso temporal
            for t in range(inputs.size(1)):
                x_t = inputs[:, t, :]  # (batch_size, input_size)
                h_t, c_t = model.lstm_cell(x_t, h_t, c_t)

            # Calcular la salida
            logits = model.fc(h_t)  # (batch_size, output_size)

            # Calcular la pérdida
            loss = criterion(logits, targets)

            epoch_loss += loss.item()

            # Cálculo de exactitud
            predicted = torch.round(torch.sigmoid(logits))  # Para BCEWithLogitsLoss
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

            # Actualizar la barra de progreso
            average_loss = epoch_loss / len(dataloader)
            accuracy = 100.0 * correct / total
            progress_bar.set_postfix(loss=average_loss, accuracy=f"{accuracy:.2f}%")

    average_loss = epoch_loss / len(dataloader)
    average_accuracy = 100.0 * correct / total
    return average_loss, average_accuracy

def save_model(model, optimizer, epoch, loss, path):
    """
    Guarda el estado del modelo y del optimizador.

    Args:
        model (nn.Module): Modelo a guardar.
        optimizer (torch.optim.Optimizer): Optimizador a guardar.
        epoch (int): Número de la época actual.
        loss (float): Pérdida actual.
        path (str): Ruta donde se guardará el modelo.
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path}")

def load_model(model, optimizer, path, device):
    """
    Carga el estado del modelo y del optimizador desde un archivo.

    Args:
        model (nn.Module): Modelo a cargar.
        optimizer (torch.optim.Optimizer): Optimizador a cargar.
        path (str): Ruta desde donde se cargará el modelo.
        device (torch.device): Dispositivo (CPU o GPU).

    Returns:
        tuple: (época desde la cual se reanudó el entrenamiento, pérdida en la última época registrada)
    """
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

def main():
    # Parámetros
    input_size = 10        # Tamaño del vector de entrada
    hidden_size = 20       # Tamaño del estado oculto
    sequence_length = 5    # Longitud de la secuencia
    batch_size = 32        # Tamaño del lote
    num_epochs = 5         # Número de épocas
    learning_rate = 0.001  # Tasa de aprendizaje
    save_path = 'lstm_cell.pth'  # Ruta para guardar el modelo

    # Crear datos de ejemplo (para demostración; usa datos reales en práctica)
    num_samples = 1000
    sequences = []
    labels = []
    for _ in range(num_samples):
        seq = torch.randn(sequence_length, input_size).tolist()  # Secuencia aleatoria
        label = random.randint(0, 1)  # Etiqueta aleatoria (por ejemplo, clasificación binaria)
        sequences.append(seq)
        labels.append(label)

    # Crear el dataset y el dataloader
    dataset = SequenceDataset(sequences, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Inicializar el modelo, función de pérdida y optimizador
    model = LSTMModel(input_size, hidden_size, output_size=1).to(device)
    criterion = nn.BCEWithLogitsLoss()  # Función de pérdida para clasificación binaria
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(save_path):
        start_epoch, _ = load_model(model, optimizer, save_path, device)

    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        print(f"\nEpoca {epoch+1}/{num_epochs}")
        train(model, dataloader, criterion, optimizer, device, epoch, num_epochs)
        val_loss, val_accuracy = evaluate(model, dataloader, criterion, device)
        print(f"Validación - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch + 1, val_loss, save_path)

    # Ejemplo de uso: realizar una predicción
    model.eval()
    with torch.no_grad():
        # Generar una secuencia de entrada aleatoria
        test_seq = torch.randn(1, sequence_length, input_size).to(device)
        logits = model(test_seq)  # (1, 1)
        # Aplicar una función de activación (sigmoide) para obtener la probabilidad
        prob = torch.sigmoid(logits)
        prediction = torch.round(prob)
        print(f"\nPredicción de la secuencia de prueba: {prediction.item()}, Probabilidad: {prob.item():.4f}")

if __name__ == "__main__":
    main()


**Observaciones**

* En el ejemplo proporcionado, se generan datos aleatorios para la demostración. Para una aplicación real, deberías reemplazar esta parte con datos auténticos que representen la tarea de clasificación que deseas realizar.
* Considera normalizar tus datos de entrada para mejorar el rendimiento del modelo.
* Además del recorte de gradientes y el dropout ya implementado, podrías explorar otras técnicas de regularización como la normalización de pesos o el uso de capas adicionales para mejorar la capacidad de generalización del modelo.
* Experimenta con diferentes tasas de aprendizaje, tamaños de lote, dimensiones de embeddings y tamaños ocultos para encontrar la mejor configuración para tu tarea específica. Herramientas como Optuna o Ray Tune pueden ayudarte a automatizar la búsqueda de hiperparámetros.
* Si trabajas con secuencias de longitud variable, considera implementar técnicas de padding y usar pack_padded_sequence y pad_packed_sequence de PyTorch para manejar eficientemente las secuencias de diferentes longitudes.

### NMT

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os
import random
import numpy as np

# Establecer semillas para reproducibilidad
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Verificar dispositivo (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Crear un vocabulario de juguete para Inglés y Español
source_vocab = {'<pad>':0, 'the':1, 'green':2, 'witch':3, 'arrived':4}
target_vocab = {'<pad>':0, '<s>':1, 'llegó':2, 'la':3, 'bruja':4, 'verde':5, '</s>':6}
source_vocab_size = len(source_vocab)  # 5
target_vocab_size = len(target_vocab)  # 7

# Diccionarios inversos para decodificación
source_vocab_inv = {v: k for k, v in source_vocab.items()}
target_vocab_inv = {v: k for k, v in target_vocab.items()}

# Convertir oraciones a secuencias de IDs de tokens
src_sentence = [source_vocab[word] for word in ["the", "green", "witch", "arrived"]]
trg_sentence = [target_vocab['<s>']] + [target_vocab[word] for word in ["llegó", "la", "bruja", "verde", "</s>"]]

# Definir parámetros del modelo
embed_size = 32
hidden_size = 64
learning_rate = 0.001

# Modelo de Encoder
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embed_size, padding_idx=0)
        self.rnn = nn.GRU(embed_size, hidden_size, batch_first=True)
        
    def forward(self, x):
        embedded = self.embedding(x)  # [batch_size, seq_length, embed_size]
        outputs, hidden = self.rnn(embedded)  # outputs: [batch_size, seq_length, hidden_size], hidden: [1, batch_size, hidden_size]
        return outputs, hidden

# Modelo de Decoder
class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_size, embed_size, padding_idx=0)
        self.rnn = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden):
        x = x.unsqueeze(1)  # [batch_size] -> [batch_size, 1]
        embedded = self.embedding(x)  # [batch_size, 1, embed_size]
        output, hidden = self.rnn(embedded, hidden)  # output: [batch_size, 1, hidden_size]
        prediction = self.fc(output.squeeze(1))  # [batch_size, output_size]
        return prediction, hidden

# Modelo NMT con encoder y decoder
class NMTModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(NMTModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        Args:
            src: [batch_size, src_len]
            trg: [batch_size, trg_len]
            teacher_forcing_ratio: probabilidad de usar el valor real como siguiente input en lugar de la predicción
        Returns:
            outputs: [batch_size, trg_len, output_dim]
        """
        batch_size = trg.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc.out_features
        
        # Inicializar el tensor de outputs
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(src.device)
        
        # Obtener las salidas del encoder
        encoder_outputs, hidden = self.encoder(src)
        
        # El primer input al decoder es el token <s>
        input = trg[:,0]  # [batch_size]
        
        for t in range(1, trg_len):
            # Pasar el input y el estado oculto actual al decoder
            output, hidden = self.decoder(input, hidden)
            # Guardar la predicción
            outputs[:, t] = output
            # Decidir si usar teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio
            # Obtener la siguiente entrada
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1
        
        return outputs

# Dataset personalizado para pares de oración
class TranslationDataset(Dataset):
    def __init__(self, src_sentences, trg_sentences):
        """
        Args:
            src_sentences: list of list of int
            trg_sentences: list of list of int
        """
        assert len(src_sentences) == len(trg_sentences), "Las oraciones fuente y destino deben tener la misma longitud."
        self.src_sentences = src_sentences
        self.trg_sentences = trg_sentences
    
    def __len__(self):
        return len(self.src_sentences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.src_sentences[idx], dtype=torch.long), torch.tensor(self.trg_sentences[idx], dtype=torch.long)

# Crear múltiples copias del mismo par de oraciones para simular datos
num_samples = 1000
src_sentences = [src_sentence for _ in range(num_samples)]
trg_sentences = [trg_sentence for _ in range(num_samples)]

# Crear el dataset y el dataloader
dataset = TranslationDataset(src_sentences, trg_sentences)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Instanciar modelos, optimizador y función de pérdida
encoder = Encoder(source_vocab_size, embed_size, hidden_size)
decoder = Decoder(target_vocab_size, embed_size, hidden_size)
model = NMTModel(encoder, decoder).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=0)

# Función de entrenamiento
def train_epoch(model, dataloader, criterion, optimizer, device, epoch, total_epochs, clip=1.0):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{total_epochs}")
    
    for batch_idx, (src, trg) in progress_bar:
        src = src.to(device)  # [batch_size, src_len]
        trg = trg.to(device)  # [batch_size, trg_len]
        
        optimizer.zero_grad()
        
        # Obtener las predicciones del modelo
        output = model(src, trg)  # [batch_size, trg_len, trg_vocab_size]
        
        # Reorganizar las salidas para calcular la pérdida
        output_dim = output.shape[-1]
        output = output[:,1:].reshape(-1, output_dim)  # [batch_size * (trg_len -1), trg_vocab_size]
        trg = trg[:,1:].reshape(-1)  # [batch_size * (trg_len -1)]
        
        # Calcular la pérdida
        loss = criterion(output, trg)
        loss.backward()
        
        # Recorte de gradientes para evitar explosiones
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        # Cálculo de exactitud
        predicted = output.argmax(1)  # [batch_size * (trg_len -1)]
        correct += (predicted == trg).sum().item()
        total += trg.size(0)
        
        # Actualizar la barra de progreso
        average_loss = epoch_loss / (batch_idx +1)
        accuracy = 100.0 * correct / total
        progress_bar.set_postfix(loss=average_loss, accuracy=f"{accuracy:.2f}%")
    
    print(f"Epoch {epoch+1}/{total_epochs} - Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Función de evaluación
def evaluate_epoch(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(dataloader, desc="Evaluando", leave=False)
    with torch.no_grad():
        for src, trg in progress_bar:
            src = src.to(device)  # [batch_size, src_len]
            trg = trg.to(device)  # [batch_size, trg_len]
            
            # Obtener las predicciones del modelo
            output = model(src, trg, teacher_forcing_ratio=0.0)  # [batch_size, trg_len, trg_vocab_size]
            
            # Reorganizar las salidas para calcular la pérdida
            output_dim = output.shape[-1]
            output = output[:,1:].reshape(-1, output_dim)  # [batch_size * (trg_len -1), trg_vocab_size]
            trg = trg[:,1:].reshape(-1)  # [batch_size * (trg_len -1)]
            
            # Calcular la pérdida
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
            # Cálculo de exactitud
            predicted = output.argmax(1)  # [batch_size * (trg_len -1)]
            correct += (predicted == trg).sum().item()
            total += trg.size(0)
            
            # Actualizar la barra de progreso
            average_loss = epoch_loss / len(dataloader)
            accuracy = 100.0 * correct / total
            progress_bar.set_postfix(loss=average_loss, accuracy=f"{accuracy:.2f}%")
    
    average_loss = epoch_loss / len(dataloader)
    average_accuracy = 100.0 * correct / total
    return average_loss, average_accuracy

# Función para guardar el modelo
def save_model(model, optimizer, epoch, loss, path):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path}")

# Función para cargar el modelo
def load_model(model, optimizer, path, device):
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

# Función de traducción (inferencia)
def translate(model, src_sentence, target_vocab_inv, device, max_len=10):
    model.eval()
    src_tensor = torch.LongTensor([src_sentence]).to(device)  # [1, src_len]
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)
        input = torch.LongTensor([target_vocab['<s>']]).to(device)  # [1]
        translated_sentence = []
        
        for _ in range(max_len):
            output, hidden = model.decoder(input, hidden)  # output: [1, target_vocab_size]
            top1 = output.argmax(1).item()
            if top1 == target_vocab['</s>']:
                break
            translated_sentence.append(target_vocab_inv[top1])
            input = torch.LongTensor([top1]).to(device)
    
    return ' '.join(translated_sentence)

def main():
    # Parámetros
    embed_size = 32
    hidden_size = 64
    learning_rate = 0.001
    num_epochs = 1000
    save_path = 'nmt_model.pth'
    
    # Crear datos de ejemplo (usando múltiples copias de la misma oración para simular datos)
    num_samples = 1000
    src_sentences = [src_sentence for _ in range(num_samples)]
    trg_sentences = [trg_sentence for _ in range(num_samples)]
    
    # Crear el dataset y el dataloader
    dataset = TranslationDataset(src_sentences, trg_sentences)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Instanciar modelos, optimizador y función de pérdida
    encoder = Encoder(source_vocab_size, embed_size, hidden_size)
    decoder = Decoder(target_vocab_size, embed_size, hidden_size)
    model = NMTModel(encoder, decoder).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignora el padding
    
    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(save_path):
        start_epoch, _ = load_model(model, optimizer, save_path, device)
    
    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        print(f"\nEpoca {epoch+1}/{num_epochs}")
        train_epoch(model, dataloader, criterion, optimizer, device, epoch, num_epochs)
        val_loss, val_accuracy = evaluate_epoch(model, dataloader, criterion, device)
        print(f"Validación - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")
    
        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch + 1, val_loss, save_path)
    
    # Ejemplo de uso: realizar una predicción
    translated_sentence = translate(model, src_sentence, target_vocab_inv, device)
    print("Oracion traducida:", translated_sentence)

if __name__ == "__main__":
    main()


**Observaciones**

- Dado que  se está utilizando datos sintéticos (múltiples copias de un solo par de oraciones), el modelo puede aprender rápidamente a mapear correctamente las entradas a las salidas. Sin embargo, en aplicaciones reales, necesitarías un conjunto de datos mucho más grande y variado para que el modelo generalice bien.
- Para aplicaciones prácticas, considera utilizar conjuntos de datos reales como IWSLT o WMT para entrenar tu modelo de traducción.
- Si trabajas con secuencias de diferentes longitudes, implementa técnicas de padding y utiliza pack_padded_sequence y pad_packed_sequence de PyTorch para optimizar el procesamiento.
- Experimenta con diferentes tasas de aprendizaje, tamaños de embedding, tamaños ocultos, y proporciones de teacher forcing para mejorar el rendimiento del modelo.
- Además de la pérdida y la exactitud, considera implementar métricas como BLEU para evaluar la calidad de las traducciones
- Explora arquitecturas más avanzadas como Transformer, que han demostrado un rendimiento superior en tareas de traducción automática.
- Implementa técnicas de regularización como dropout para evitar el sobreajuste, especialmente cuando trabajas con conjuntos de datos más grandes.

### Mecanismo de atención

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Barra de progreso
import os
import random
import numpy as np
import matplotlib.pyplot as plt

# Establecer semillas para reproducibilidad
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Verificar dispositivo (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Crear un vocabulario de juguete para Inglés y Español
source_vocab = {'<pad>':0, '<s>':1, '</s>':2, 'the':3, 'green':4, 'witch':5, 'arrived':6}
target_vocab = {'<pad>':0, '<s>':1, '</s>':2, 'llegó':3, 'la':4, 'bruja':5, 'verde':6}
source_vocab_size = len(source_vocab)  # 7
target_vocab_size = len(target_vocab)  # 7

# Diccionarios inversos para decodificación
source_vocab_inv = {v: k for k, v in source_vocab.items()}
target_vocab_inv = {v: k for k, v in target_vocab.items()}

# Convertir oraciones a secuencias de IDs de tokens
def tokenize(sentence, vocab):
    return [vocab['<s>']] + [vocab[word] for word in sentence.split()] + [vocab['</s>']]

src_sentence = "the green witch arrived"
trg_sentence = "llegó la bruja verde"

src_ids = tokenize(src_sentence, source_vocab)  # [1, 3, 4, 5, 6, 2]
trg_ids = tokenize(trg_sentence, target_vocab)  # [1, 3, 4, 5, 6, 2]

print(f"Source IDs: {src_ids}")
print(f"Target IDs: {trg_ids}")

# Modelo de Encoder
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size):
        """
        Encoder con GRU.

        Args:
            input_size (int): Tamaño del vocabulario de entrada.
            embed_size (int): Tamaño de los embeddings.
            hidden_size (int): Tamaño del estado oculto de la GRU.
        """
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embed_size, padding_idx=0)
        self.rnn = nn.GRU(embed_size, hidden_size, batch_first=True)
        
    def forward(self, x):
        """
        Pase hacia adelante del encoder.

        Args:
            x (Tensor): Secuencia de entrada, forma (batch_size, seq_length).

        Returns:
            outputs (Tensor): Salidas de todos los pasos temporales, forma (batch_size, seq_length, hidden_size).
            hidden (Tensor): Último estado oculto, forma (1, batch_size, hidden_size).
        """
        embedded = self.embedding(x)  # [batch_size, seq_length, embed_size]
        outputs, hidden = self.rnn(embedded)  # outputs: [batch_size, seq_length, hidden_size], hidden: [1, batch_size, hidden_size]
        return outputs, hidden

# Mecanismo de Atención
class Attention(nn.Module):
    def __init__(self, hidden_size):
        """
        Mecanismo de atención.

        Args:
            hidden_size (int): Tamaño del estado oculto.
        """
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)
        
    def forward(self, hidden, encoder_outputs):
        """
        Cálculo de pesos de atención.

        Args:
            hidden (Tensor): Estado oculto actual del decoder, forma (batch_size, hidden_size).
            encoder_outputs (Tensor): Salidas del encoder, forma (batch_size, seq_len, hidden_size).

        Returns:
            attention (Tensor): Pesos de atención normalizados, forma (batch_size, seq_len).
        """
        batch_size = encoder_outputs.size(0)
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)  # [batch_size, seq_len, hidden_size]
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch_size, seq_len, hidden_size]
        attention = self.v(energy).squeeze(2)  # [batch_size, seq_len]
        return torch.softmax(attention, dim=1)  # [batch_size, seq_len]

# Modelo de Decoder con Atención
class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size):
        """
        Decoder con atención y GRU.

        Args:
            output_size (int): Tamaño del vocabulario de salida.
            embed_size (int): Tamaño de los embeddings.
            hidden_size (int): Tamaño del estado oculto de la GRU.
        """
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_size, embed_size, padding_idx=0)
        self.attention = Attention(hidden_size)
        self.rnn = nn.GRU(hidden_size + embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        
    def forward(self, x, hidden, encoder_outputs):
        """
        Pase hacia adelante del decoder.

        Args:
            x (Tensor): Token actual, forma (batch_size).
            hidden (Tensor): Estado oculto anterior, forma (batch_size, hidden_size).
            encoder_outputs (Tensor): Salidas del encoder, forma (batch_size, seq_len, hidden_size).

        Returns:
            output (Tensor): Logits para el siguiente token, forma (batch_size, output_size).
            hidden (Tensor): Nuevo estado oculto, forma (batch_size, hidden_size).
            attn_weights (Tensor): Pesos de atención, forma (batch_size, seq_len).
        """
        embedded = self.embedding(x)  # [batch_size, embed_size]
        attn_weights = self.attention(hidden, encoder_outputs)  # [batch_size, seq_len]
        attn_applied = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)  # [batch_size, 1, hidden_size]
        attn_applied = attn_applied.squeeze(1)  # [batch_size, hidden_size]
        
        rnn_input = torch.cat((embedded, attn_applied), dim=1).unsqueeze(1)  # [batch_size, 1, embed_size + hidden_size]
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))  # output: [batch_size, 1, hidden_size]
        output = output.squeeze(1)  # [batch_size, hidden_size]
        output = self.fc(torch.cat((output, attn_applied), dim=1))  # [batch_size, output_size]
        
        return output, hidden.squeeze(0), attn_weights

# Modelo Seq2Seq con Encoder y Decoder
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        """
        Modelo Seq2Seq que integra el encoder y el decoder.

        Args:
            encoder (Encoder): Modelo de encoder.
            decoder (Decoder): Modelo de decoder.
            device (torch.device): Dispositivo (CPU/GPU).
        """
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        Pase hacia adelante del modelo Seq2Seq.

        Args:
            src (Tensor): Secuencia de entrada, forma (batch_size, src_len).
            trg (Tensor): Secuencia de salida, forma (batch_size, trg_len).
            teacher_forcing_ratio (float): Proporción de uso de teacher forcing.

        Returns:
            outputs (Tensor): Logits para cada token de la secuencia de salida, forma (batch_size, trg_len, output_size).
        """
        batch_size = trg.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc.out_features
        
        # Inicializar el tensor de salidas
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        # Obtener las salidas del encoder
        encoder_outputs, hidden = self.encoder(src)  # hidden: [1, batch_size, hidden_size]
        hidden = hidden.squeeze(0)  # [batch_size, hidden_size]
        
        # El primer input al decoder es el token <s>
        input = trg[:,0]  # [batch_size]
        
        for t in range(1, trg_len):
            output, hidden, attn_weights = self.decoder(input, hidden, encoder_outputs)
            outputs[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1
        
        return outputs

# Dataset personalizado para pares de oración
class TranslationDataset(Dataset):
    def __init__(self, src_sentences, trg_sentences):
        """
        Args:
            src_sentences (list of list of int): Lista de secuencias de entrada.
            trg_sentences (list of list of int): Lista de secuencias de salida.
        """
        assert len(src_sentences) == len(trg_sentences), "Las oraciones fuente y destino deben tener la misma longitud."
        self.src_sentences = src_sentences
        self.trg_sentences = trg_sentences
    
    def __len__(self):
        return len(self.src_sentences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.src_sentences[idx], dtype=torch.long), torch.tensor(self.trg_sentences[idx], dtype=torch.long)

# Función para visualizar los pesos de atención
def visualize_attention(attn_weights, src_sentence, trg_sentence, target_vocab_inv):
    """
    Visualiza los pesos de atención.

    Args:
        attn_weights (Tensor): Pesos de atención, forma (trg_len, src_len).
        src_sentence (list of str): Oración de entrada.
        trg_sentence (list of str): Oración de salida.
        target_vocab_inv (dict): Diccionario inverso del vocabulario de destino.
    """
    attn_weights = attn_weights.cpu().detach().numpy()
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    cax = ax.matshow(attn_weights, cmap='bone')
    fig.colorbar(cax)

    ax.set_xticklabels([''] + src_sentence, rotation=90)
    ax.set_yticklabels([''] + trg_sentence)

    ax.xaxis.set_major_locator(plt.MultipleLocator(1))
    ax.yaxis.set_major_locator(plt.MultipleLocator(1))

    plt.show()

# Función de entrenamiento por época
def train_epoch(model, dataloader, criterion, optimizer, device, epoch, total_epochs, clip=1.0):
    """
    Función de entrenamiento para una época.

    Args:
        model (Seq2Seq): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los datos de entrenamiento.
        criterion (nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        device (torch.device): Dispositivo (CPU/GPU).
        epoch (int): Número de la época actual.
        total_epochs (int): Número total de épocas.
        clip (float, opcional): Valor máximo para el recorte de gradientes. Por defecto es 1.0.
    """
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{total_epochs}")
    
    for batch_idx, (src, trg) in progress_bar:
        src = src.to(device)  # [batch_size, src_len]
        trg = trg.to(device)  # [batch_size, trg_len]
        
        optimizer.zero_grad()
        
        # Obtener las predicciones del modelo
        output = model(src, trg)  # [batch_size, trg_len, output_size]
        
        # Reorganizar las salidas para calcular la pérdida
        output_dim = output.shape[-1]
        output = output[:,1:].reshape(-1, output_dim)  # [batch_size * (trg_len -1), output_size]
        trg = trg[:,1:].reshape(-1)  # [batch_size * (trg_len -1)]
        
        # Calcular la pérdida
        loss = criterion(output, trg)
        loss.backward()
        
        # Recorte de gradientes para evitar explosiones
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        # Cálculo de exactitud
        predicted = output.argmax(1)  # [batch_size * (trg_len -1)]
        correct += (predicted == trg).sum().item()
        total += trg.size(0)
        
        # Actualizar la barra de progreso
        average_loss = epoch_loss / (batch_idx +1)
        accuracy = 100.0 * correct / total
        progress_bar.set_postfix(loss=average_loss, accuracy=f"{accuracy:.2f}%")
    
    print(f"Epoch {epoch+1}/{total_epochs} - Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Función de evaluación por época
def evaluate_epoch(model, dataloader, criterion, device):
    """
    Función de evaluación para calcular la pérdida y exactitud en el conjunto de validación.

    Args:
        model (Seq2Seq): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los datos de validación.
        criterion (nn.Module): Función de pérdida.
        device (torch.device): Dispositivo (CPU/GPU).

    Returns:
        float: Pérdida promedio en el conjunto de validación.
        float: Exactitud promedio en el conjunto de validación.
    """
    model.eval()
    epoch_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(dataloader, desc="Evaluando", leave=False)
    with torch.no_grad():
        for src, trg in progress_bar:
            src = src.to(device)  # [batch_size, src_len]
            trg = trg.to(device)  # [batch_size, trg_len]
            
            # Obtener las predicciones del modelo
            output = model(src, trg, teacher_forcing_ratio=0.0)  # [batch_size, trg_len, output_size]
            
            # Reorganizar las salidas para calcular la pérdida
            output_dim = output.shape[-1]
            output = output[:,1:].reshape(-1, output_dim)  # [batch_size * (trg_len -1), output_size]
            trg = trg[:,1:].reshape(-1)  # [batch_size * (trg_len -1)]
            
            # Calcular la pérdida
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
            # Cálculo de exactitud
            predicted = output.argmax(1)  # [batch_size * (trg_len -1)]
            correct += (predicted == trg).sum().item()
            total += trg.size(0)
            
            # Actualizar la barra de progreso
            average_loss = epoch_loss / len(dataloader)
            accuracy = 100.0 * correct / total
            progress_bar.set_postfix(loss=average_loss, accuracy=f"{accuracy:.2f}%")
    
    average_loss = epoch_loss / len(dataloader)
    average_accuracy = 100.0 * correct / total
    return average_loss, average_accuracy

# Función para guardar el modelo
def save_model(model, optimizer, epoch, loss, path):
    """
    Guarda el estado del modelo y del optimizador.

    Args:
        model (Seq2Seq): Modelo a guardar.
        optimizer (torch.optim.Optimizer): Optimizador a guardar.
        epoch (int): Número de la época actual.
        loss (float): Pérdida actual.
        path (str): Ruta donde se guardará el modelo.
    """
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(state, path)
    print(f"Modelo guardado en {path}")

# Función para cargar el modelo
def load_model(model, optimizer, path, device):
    """
    Carga el estado del modelo y del optimizador desde un archivo.

    Args:
        model (Seq2Seq): Modelo a cargar.
        optimizer (torch.optim.Optimizer): Optimizador a cargar.
        path (str): Ruta desde donde se cargará el modelo.
        device (torch.device): Dispositivo (CPU/GPU).

    Returns:
        tuple: (época desde la cual se reanudó el entrenamiento, pérdida en la última época registrada)
    """
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Modelo cargado desde {path} (Época {epoch}, Pérdida {loss})")
        return epoch, loss
    else:
        print(f"No se encontró el archivo {path}. Se iniciará el entrenamiento desde cero.")
        return 0, None

# Función de traducción (inferencia)
def translate_sentence(model, src_sentence, source_vocab, target_vocab_inv, device, max_len=10):
    """
    Traduce una oración fuente utilizando el modelo entrenado.

    Args:
        model (Seq2Seq): Modelo entrenado.
        src_sentence (str): Oración de entrada en inglés.
        source_vocab (dict): Vocabulario de fuente (inglés).
        target_vocab_inv (dict): Vocabulario inverso de destino (español).
        device (torch.device): Dispositivo (CPU/GPU).
        max_len (int, opcional): Longitud máxima de la traducción. Por defecto es 10.

    Returns:
        str: Oración traducida en español.
    """
    model.eval()
    tokens = src_sentence.split()
    src_ids = [source_vocab['<s>']] + [source_vocab.get(word, source_vocab['<pad>']) for word in tokens] + [source_vocab['</s>']]
    src_tensor = torch.LongTensor(src_ids).unsqueeze(0).to(device)  # [1, src_len]
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)
        hidden = hidden.squeeze(0)  # [1, hidden_size]
        input = torch.LongTensor([target_vocab['<s>']]).to(device)  # [1]
        translated_sentence = []
        
        for _ in range(max_len):
            output, hidden, attn_weights = model.decoder(input, hidden, encoder_outputs)
            top1 = output.argmax(1).item()
            if top1 == target_vocab['</s>']:
                break
            translated_sentence.append(target_vocab_inv[top1])
            input = torch.LongTensor([top1]).to(device)
    
    return ' '.join(translated_sentence)

# Visualizar pesos de atención para una traducción específica
def visualize_attention_weights(model, src_sentence, trg_sentence, source_vocab_inv, target_vocab_inv, device):
    """
    Visualiza los pesos de atención para una oración específica.

    Args:
        model (Seq2Seq): Modelo entrenado.
        src_sentence (str): Oración de entrada en inglés.
        trg_sentence (str): Oración de salida en español.
        source_vocab_inv (dict): Vocabulario inverso de fuente.
        target_vocab_inv (dict): Vocabulario inverso de destino.
        device (torch.device): Dispositivo (CPU/GPU).
    """
    model.eval()
    tokens = src_sentence.split()
    src_ids = [source_vocab['<s>']] + [source_vocab.get(word, source_vocab['<pad>']) for word in tokens] + [source_vocab['</s>']]
    src_tensor = torch.LongTensor(src_ids).unsqueeze(0).to(device)  # [1, src_len]
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)
        hidden = hidden.squeeze(0)  # [1, hidden_size]
        input = torch.LongTensor([target_vocab['<s>']]).to(device)  # [1]
        translated_sentence = []
        attn_weights_all = []
        
        for _ in range(len(trg_sentence.split()) + 2):  # +2 for <s> and </s>
            output, hidden, attn_weights = model.decoder(input, hidden, encoder_outputs)
            top1 = output.argmax(1).item()
            if top1 == target_vocab['</s>']:
                break
            translated_sentence.append(target_vocab_inv[top1])
            attn_weights_all.append(attn_weights.cpu().numpy())
            input = torch.LongTensor([top1]).to(device)
    
    # Convertir a matriz de atención
    attn_matrix = np.stack(attn_weights_all, axis=0)  # [trg_len, src_len]
    
    # Obtener las palabras
    src_words = [source_vocab_inv[id] for id in src_ids]
    trg_words = [target_vocab_inv[id] for id in [target_vocab['<s>']] + [target_vocab.get(word, target_vocab['<pad>']) for word in translated_sentence] + [target_vocab['</s>']]]
    
    # Visualizar
    fig, ax = plt.subplots(figsize=(10,10))
    cax = ax.matshow(attn_matrix, cmap='bone')
    fig.colorbar(cax)

    ax.set_xticklabels([''] + src_words, rotation=90)
    ax.set_yticklabels([''] + translated_sentence)

    ax.xaxis.set_major_locator(plt.MultipleLocator(1))
    ax.yaxis.set_major_locator(plt.MultipleLocator(1))

    plt.show()

# Función principal
def main():
    # Parámetros
    embed_size = 256
    hidden_size = 512
    learning_rate = 0.001
    num_epochs = 1000
    save_path = 'seq2seq_attention.pth'
    
    # Crear datos de ejemplo (usando múltiples copias del par de oraciones para simular datos)
    num_samples = 1000
    src_sentences = [src_ids for _ in range(num_samples)]
    trg_sentences = [trg_ids for _ in range(num_samples)]
    
    # Crear el dataset y el dataloader
    dataset = TranslationDataset(src_sentences, trg_sentences)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Instanciar modelos, optimizador y función de pérdida
    encoder = Encoder(source_vocab_size, embed_size, hidden_size)
    decoder = Decoder(target_vocab_size, embed_size, hidden_size)
    model = Seq2Seq(encoder, decoder, device).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=source_vocab['<pad>'])
    
    # Opcional: Cargar un modelo previamente guardado
    start_epoch = 0
    if os.path.exists(save_path):
        start_epoch, _ = load_model(model, optimizer, save_path, device)
    
    # Ciclo de entrenamiento
    for epoch in range(start_epoch, num_epochs):
        print(f"\nEpoca {epoch+1}/{num_epochs}")
        train_epoch(model, dataloader, criterion, optimizer, device, epoch, num_epochs)
        val_loss, val_accuracy = evaluate_epoch(model, dataloader, criterion, device)
        print(f"Validación - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")
        
        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch + 1, val_loss, save_path)
        
        # Opcional: Visualizar atención cada 100 épocas
        if (epoch +1) % 100 == 0:
            visualize_attention_weights(model, src_sentence, trg_sentence, source_vocab_inv, target_vocab_inv, device)
    
    # Realizar una traducción de ejemplo
    translated = translate_sentence(model, src_sentence, source_vocab, target_vocab_inv, device)
    print("\nTraducción de ejemplo:")
    print(f"Fuente: {src_sentence}")
    print(f"Traducción: {translated}")
    
    # Visualizar los pesos de atención para la traducción de ejemplo
    visualize_attention_weights(model, src_sentence, translated, source_vocab_inv, target_vocab_inv, device)

if __name__ == "__main__":
    main()


Al ejecutar el código mejorado, el modelo se entrenará para traducir la oración "the green witch arrived" al español "llegó la bruja verde". Además, cada 100 épocas, se visualizarán los pesos de atención para entender cómo el modelo está prestando atención a las palabras de entrada durante la traducción.


Durante el entrenamiento, deberías ver una salida similar a la siguiente cada 10 épocas:

```
Usando dispositivo: cpu

Epoca 1/1000
Epoch 1/1000: 100%|█████████████████████████████████████| 31/31 [00:00<00:00,  9.95it/s]
Epoch 1/1000 - Loss: 1.9452, Accuracy: 32.00%
Validación - Loss: 1.9345, Accuracy: 33.20%

Modelo guardado en seq2seq_attention.pth
```

Al final del entrenamiento, se realizará una traducción de ejemplo y se visualizarán los pesos de atención:

```
Traducción de ejemplo:
Fuente: the green witch arrived
Traducción: llegó la bruja verde

```


**Observaciones**

- Actualmente, el conjunto de datos está compuesto por múltiples copias del mismo par de oraciones. Para mejorar la capacidad de generalización del modelo, es recomendable utilizar un conjunto de datos más amplio y variado.
- Si trabajas con secuencias de diferentes longitudes, considera implementar técnicas de padding y utilizar `pack_padded_sequence` y `pad_packed_sequence` de PyTorch para optimizar el procesamiento.
- Experimenta con diferentes tamaños de embedding, tamaños ocultos, tasas de aprendizaje y proporciones de teacher forcing para encontrar la mejor configuración para tu tarea específica.
- Además de la pérdida y la precisión, considera implementar métricas como BLEU para evaluar la calidad de las traducciones de manera más exhaustiva.
- Para evitar el sobreajuste y reducir el tiempo de entrenamiento, puedes implementar una técnica de early stopping que detenga el entrenamiento cuando la pérdida de validación deje de mejorar.
- Explora arquitecturas más avanzadas como los Transformers, que han demostrado un rendimiento superior en tareas de traducción automática.
- Implementa técnicas de regularización como Dropout para mejorar la capacidad de generalización del modelo.





### Ejercicios adicionales

#### 1. **Ampliar el vocabulario y el conjunto de datos**

**Objetivo**: Incrementar la capacidad del modelo para manejar una variedad mayor de palabras y frases.

**Tareas**:
- **Agregar más palabras**: Amplía los vocabularios `source_vocab` y `target_vocab` con más palabras relevantes.
- **Crear más pares de oraciones**: Genera múltiples pares de oraciones en inglés y español que contengan las nuevas palabras añadidas.
- **Actualizar el dataset**: Modifica la clase `TranslationDataset` para manejar el nuevo conjunto de datos.

**Beneficios**: Mejora la capacidad del modelo para generalizar y manejar traducciones más complejas.

#### 2. **Implementar un encoder bidireccional**

**Objetivo**: Mejorar la capacidad del encoder para capturar información contextual de ambas direcciones de la secuencia de entrada.

**Tareas**:
- **Modificar el encoder**: Cambia el GRU del encoder para que sea bidireccional añadiendo el parámetro `bidirectional=True`.
- **Ajustar el decoder**: Actualiza el decoder para manejar el estado oculto bidireccional concatenado o sumado.
- **Actualizar la función de atención**: Asegúrate de que la atención maneje las salidas bidireccionales correctamente.

**Beneficios**: Captura una comprensión más rica de la secuencia de entrada, lo que puede mejorar la precisión de la traducción.

#### 3. **Manejo de secuencias de longitud variable con padding y máscaras**

**Objetivo**: Permitir que el modelo maneje secuencias de diferentes longitudes de manera eficiente.

**Tareas**:
- **Implementar padding**: Añade padding a las secuencias más cortas para igualar la longitud máxima en el batch.
- **Crear máscaras de padding**: Genera máscaras que indiquen qué posiciones son padding para evitar que el modelo preste atención a ellas.
- **Actualizar el modelo**: Modifica el encoder y el mecanismo de atención para utilizar las máscaras y ignorar el padding.

**Beneficios**: Permite trabajar con conjuntos de datos reales donde las secuencias de entrada y salida varían en longitud.

#### 4. **Implementar el beam search para inferencia**

**Objetivo**: Mejorar la calidad de las traducciones generadas durante la inferencia al considerar múltiples posibles secuencias de salida.

**Tareas**:
- **Crear la función de beam search**: Implementa una función que explore múltiples secuencias de salida simultáneamente.
- **Integrar con el modelo**: Reemplaza la traducción greedy con la búsqueda en haz en la función de traducción.
- **Ajustar parámetros**: Experimenta con diferentes tamaños de haz (beam sizes) para encontrar el equilibrio entre calidad y eficiencia.

**Beneficios**: Genera traducciones más precisas y coherentes al considerar múltiples opciones durante la generación.

#### 5. **Visualizar los pesos de atención para múltiples oraciones**

**Objetivo**: Comprender cómo el modelo presta atención a diferentes partes de la oración de entrada durante la traducción.

**Tareas**:
- **Modificar la función de visualización**: Permite la visualización de pesos de atención para múltiples pares de oraciones.
- **Crear múltiples casos de prueba**: Traduce diferentes oraciones y visualiza los pesos de atención correspondientes.
- **Interpretar los resultados**: Analiza cómo varían los pesos de atención según las oraciones de entrada.

**Beneficios**: Facilita la interpretación del comportamiento del modelo y ayuda a identificar posibles mejoras.

#### 6. **Agregar dropout para regularización**

**Objetivo**: Prevenir el sobreajuste del modelo y mejorar su capacidad de generalización.

**Tareas**:
- **Incorporar dropout en el encoder y decoder**: Añade capas de Dropout después de las capas de embedding y antes de las capas de GRU.
- **Experimentar con diferentes tasas de dropout**: Prueba diferentes valores de dropout para encontrar el óptimo.
- **Evaluar el impacto**: Observa cómo afecta el dropout a la pérdida de entrenamiento y a la precisión.

**Beneficios**: Mejora la robustez del modelo y evita que memorice el conjunto de entrenamiento.

#### 7. **Implementar early stopping**

**Objetivo**: Evitar el sobreentrenamiento deteniendo el entrenamiento cuando la pérdida de validación deja de mejorar.

**Tareas**:
- **Monitorear la pérdida de validación**: Durante el entrenamiento, observa la pérdida en el conjunto de validación.
- **Definir un criterio de detención**: Establece un número de épocas sin mejora antes de detener el entrenamiento.
- **Implementar la lógica de early stopping**: Modifica el ciclo de entrenamiento para incluir la lógica de detención.

**Beneficios**: Reduce el tiempo de entrenamiento y previene el sobreajuste.

#### 8. **Evaluar el modelo con la métrica BLEU**

**Objetivo**: Medir la calidad de las traducciones generadas de manera más exhaustiva.

**Tareas**:
- **Implementar el cálculo de BLEU**: Utiliza bibliotecas como `nltk` para calcular el puntaje BLEU de las traducciones.
- **Integrar con el ciclo de evaluación**: Añade el cálculo de BLEU durante las fases de evaluación.
- **Interpretar los resultados**: Analiza cómo varía el puntaje BLEU con diferentes configuraciones del modelo.

**Beneficios**: Proporciona una medida estándar de la calidad de las traducciones, más allá de la pérdida y la precisión.

#### 9. **Experimentar con diferentes mecanismos de atención**

**Objetivo**: Comprender cómo diferentes mecanismos de atención afectan el rendimiento del modelo.

**Tareas**:
- **Implementar atención dot-product**: Crea una variante del mecanismo de atención que utiliza productos punto.
- **Comparar con atención aditiva**: Compara el rendimiento entre atención aditiva (tu implementación actual) y dot-product.
- **Evaluar el impacto**: Observa cómo cambian la pérdida y la precisión con diferentes mecanismos de atención.

**Beneficios**: Permite identificar qué tipo de atención es más adecuada para tu tarea específica.

#### 10. **Guardar y cargar el modelo para inferencia posterior**

**Objetivo**: Permitir el uso del modelo entrenado en futuras sesiones sin necesidad de reentrenarlo.

**Tareas**:
- **Implementar funciones de guardado y carga**: Asegúrate de que el modelo y el optimizador se puedan guardar y cargar correctamente.
- **Realizar la inferencia con el modelo guardado**: Guarda el modelo después del entrenamiento y cárgalo para traducir nuevas oraciones.
- **Verificar la consistencia**: Asegúrate de que el modelo cargado produce las mismas traducciones que antes de guardarlo.

**Beneficios**: Facilita la reutilización del modelo y la realización de inferencias en diferentes contextos.

#### 11. **Optimización de hiperparámetros**

**Objetivo**: Mejorar el rendimiento del modelo ajustando los hiperparámetros clave.

**Tareas**:
- **Definir hiperparámetros a ajustar**: Por ejemplo, tamaño de embedding, tamaño oculto, tasa de aprendizaje, proporción de teacher forcing.
- **Realizar pruebas sistemáticas**: Prueba diferentes combinaciones de hiperparámetros y registra los resultados.
- **Utilizar herramientas de búsqueda**: Considera usar bibliotecas como `Optuna` o `Ray Tune` para automatizar la búsqueda de hiperparámetros.
- **Identificar la mejor configuración**: Selecciona la combinación de hiperparámetros que proporciona el mejor rendimiento.

**Beneficios**: Optimiza el modelo para obtener mejores resultados en la tarea de traducción.

#### 12. **Implementar un decoder bidireccional**

**Objetivo**: Explorar cómo un decoder bidireccional podría influir en la generación de secuencias.

**Tareas**:
- **Modificar el decoder**: Cambia el GRU del decoder para que sea bidireccional.
- **Ajustar el mecanismo de atención**: Asegúrate de que la atención maneje las salidas bidireccionales correctamente.
- **Evaluar el impacto**: Observa cómo afecta esto a la calidad de las traducciones.

**Nota**: Aunque los decoders bidireccionales no son comunes en modelos Seq2Seq tradicionales, este ejercicio es útil para comprender las implicaciones de las arquitecturas bidireccionales en ambos componentes.

**Beneficios**: Amplía tu comprensión de las arquitecturas bidireccionales y su impacto en las tareas de generación de secuencias.

#### 13. **Implementar el mecanismo de atención de Bahdanau y Luong**

**Objetivo**: Profundizar en diferentes tipos de mecanismos de atención y comparar sus efectos.

**Tareas**:
- **Atención de Bahdanau**: Implementa la atención aditiva propuesta por Bahdanau et al.
- **Atención de Luong**: Implementa la atención multiplicativa propuesta por Luong et al.
- **Comparar desempeños**: Entrena modelos con cada tipo de atención y compara sus métricas de rendimiento.

**Beneficios**: Comprende las diferencias entre los mecanismos de atención y cómo afectan al modelo.

#### 14. **Implementar un modelo transformer para traducción**

**Objetivo**: Explorar una arquitectura más avanzada que ha demostrado un rendimiento superior en tareas de traducción automática.

**Tareas**:
- **Estudiar la arquitectura transformer**: Revisa los conceptos clave como auto-atención, capas de encoder y decoder, y mecanismos de posición.
- **Implementar el transformer**: Usa la clase `nn.Transformer` de PyTorch o implementa tu propia versión.
- **Comparar con Seq2Seq**: Entrena y evalúa ambos modelos en el mismo conjunto de datos y compara sus desempeños.

**Beneficios**: Familiarízate con las arquitecturas de vanguardia en procesamiento de lenguaje natural.

#### 15. **Agregar un mecanismo de copia para manejar palabras raras o desconocidas**

**Objetivo**: Mejorar la capacidad del modelo para traducir palabras raras o fuera del vocabulario mediante un mecanismo de copia.

**Tareas**:
- **Estudiar el mecanismo de copia**: Comprende cómo permite al modelo copiar directamente palabras de la entrada a la salida.
- **Implementar el mecanismo**: Modifica el decoder para que pueda decidir entre generar una palabra del vocabulario o copiar una palabra de la entrada.
- **Evaluar el impacto**: Observa cómo afecta esto a la precisión en palabras raras o desconocidas.

**Beneficios**: Mejora la capacidad del modelo para manejar vocabularios limitados y traducciones precisas.


In [ ]:
## Tus respuestas